In [1]:
#Function to update player weekly stats

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time


In [3]:
def update_player_weekly_stats():
    
    urls = ['https://www.pro-football-reference.com/players/A/',
       'https://www.pro-football-reference.com/players/B/',
       'https://www.pro-football-reference.com/players/C/',
       'https://www.pro-football-reference.com/players/D/',
       'https://www.pro-football-reference.com/players/E/',
       'https://www.pro-football-reference.com/players/F/',
       'https://www.pro-football-reference.com/players/G/',
       'https://www.pro-football-reference.com/players/H/',
       'https://www.pro-football-reference.com/players/I/',
       'https://www.pro-football-reference.com/players/J/',
       'https://www.pro-football-reference.com/players/K/',
       'https://www.pro-football-reference.com/players/L/',
       'https://www.pro-football-reference.com/players/M/',
       'https://www.pro-football-reference.com/players/N/',
        'https://www.pro-football-reference.com/players/O/',
       'https://www.pro-football-reference.com/players/P/',
       'https://www.pro-football-reference.com/players/Q/',
       'https://www.pro-football-reference.com/players/R/',
       'https://www.pro-football-reference.com/players/S/',
       'https://www.pro-football-reference.com/players/T/',
       'https://www.pro-football-reference.com/players/U/',
       'https://www.pro-football-reference.com/players/V/',
       'https://www.pro-football-reference.com/players/W/',
       'https://www.pro-football-reference.com/players/X/',
       'https://www.pro-football-reference.com/players/Y/',
       'https://www.pro-football-reference.com/players/Z/']
    
    
    l = []
    for i in range(len(urls)):
        time.sleep(5)

        url = urls[i]
        data = requests.get(url).text
        soup = BeautifulSoup(data, 'lxml')
        tags = soup.find_all('p')
        tags = tags[:-12]
       # df_dropped_last_n = df.iloc[:-n]
       # tags = tags.astype(str)
        [l.append(x) for x in tags] 
       # l.append(tags)
    
    tags_df = pd.DataFrame(l, columns = ['Player','Position'])

    test = tags_df
    #test.head()
    test = pd.DataFrame(test)
    #test.columns
    test['colB'] = test['Player'].replace('[', '')
    
    
    test['Player'] = test['Player'].astype(str)
    test['Position'] = test['Position'].astype(str)
    #test.head()
    new = test['Player'].str.split('"', expand=True)
    
    
    new = new.drop(columns = [0])

    new['Gamelog'] = 'https://www.pro-football-reference.com' + new[1] + '/gamelog/'
    #new2 = new[2].str.split()
    new['Player'] = new[2].str.replace('</a>',"")
    new['Player'] = new['Player'].str.replace('</b>',"")
    new['Player'] = new['Player'].str.replace('>',"")

    new2 = new['Player'].str.split(' ', expand=True)
    
    pos_list = ['(WR)', '(LB)', '(RB)', '(DE)', '(S)', '(OT)', '(DB)', '(OLB)', '(OL)',
           '(QB)', '(P)', '(G)', '(CB)', '(DL)', '(FS)', '(K)', '(OG)',
           '(EDGE)', '(DT)', '(TE)', '(T)', '(NT)', '(FB)', '(SS)', '(LS)',
           '(ILB)', '(OG)', '(C)', '(DT)', '(DL)']

    new2['position'] = np.where(new2[2].isin(pos_list), new2[2], 
                           np.where(new2[3].isin(pos_list), new2[3], ""))
    new2['player'] = np.where(((~new2[3].isin(pos_list)) & (new2[3].notnull())), new2[0] + " " + new2[1] + " " + new2[2] + " " + new2[3],
                         np.where(((~new2[2].isin(pos_list)) & (new2[2].notnull())), new2[0] + " " + new2[1] + " " + new2[2],
                                 new2[0] + " " + new2[1]))
    
    
    new['Player'] = new2['player']
    
    test['Gamelog'] = new['Gamelog']
    test['Player'] = new['Player']
    test['Position_2'] = new2['position']
    
    pos = test['Position'].str.split(' ', expand=True)
    
    pos['position'] = np.where((pos[1].str.contains('19')) | (pos[1].str.contains('20')),"",
                           pos[1])

    pos['playing_period'] = np.where((pos[1].str.contains('19')) | (pos[1].str.contains('20')), pos[1],
                                pos[2])
    
    test['Position_1'] = pos['position']
    test['Playing_Prd'] = pos['playing_period']
    test['Position'] = np.where(test['Position_1'] != "", test['Position_1'],
                           test['Position_2'])
    test['Position'] = test['Position'].str[1:]
    test['Position'] = test['Position'].str[:-1]
    
    play_prd = test['Playing_Prd'].str.split('-', expand=True)
    
    test['Playing_Prd_Strt'] = play_prd[0]
    test['Playing_Prd_End'] = play_prd[1]
    
    players = test[['Gamelog', 'Player','Position',
                'Playing_Prd_Strt','Playing_Prd_End']]

    
    
    players['Playing_Prd_End'] = players['Playing_Prd_End'].astype(int)
    players['Playing_Prd_Strt'] = players['Playing_Prd_Strt'].astype(int)

    players_curr = players[players['Playing_Prd_End'] >= 2012]
    
    
    players_curr = players_curr[players_curr['Position'] != 'TE-C']
    players_curr = players_curr[players_curr['Position'] != 'DE-C']

    players_curr['Position'] = players_curr['Position'].map({'SAF':'S','LT':'T','QB/TE':'TE','G,C':'G',
                                                        'WR-PR':'WR','RG':'G','RB-WR':'RB','FB-LB':'FB',
                                                        'LB-DE':'LB','RT':'T','DE-LB':'DE','DT-DE':'DT',
                                                        'NT-DE':'DT','C-G':'C','HB':'RB','LG':'G',
                                                        'T-G':'T','DE-DT':'DE','G-T':'G','G-C':'G',
                                                        'NT':'DT', 'EDGE':'DE', 'OG':'G',
                                                        'WR':'WR','LB':'LB','RB':'RB','DB':'DB','CB':'CB',
                                                        'DE':'DE','TE':'TE','DT':'DT','T':'T','QB':'QB',
                                                        'OLB':'OLB','G':'G','S':'S','C':'C','OT':'OT',
                                                        'DL':'DL','OL':'OL','K':'K','P':'P','ILB':'ILB',
                                                        'FS':'FS','SS':'SS','FB':'FB','LS':'LS'})
    
    return players_curr
    
    

In [4]:
#function to scrape/update defense weekly stats
def def_weekly_stats_scrape(first_yr, last_yr, nfl_sched):
    
    first_wk = 1
    num_years = last_yr - first_yr
    temp_df = pd.DataFrame()
    curr_yr = first_yr
    for i in range(0,num_years+1):
        if(curr_yr < 2021):
            weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
            year = [curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr]
        else:
            weeks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
            year = [curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,curr_yr,
                   curr_yr,curr_yr,curr_yr,curr_yr,curr_yr, curr_yr]
        

        curr_yr = curr_yr+1
        
        temp2 = pd.DataFrame(data = {'week':weeks,
                            'year':year})
        
    
        temp_df = temp_df.append(temp2)
    
    temp_df['page'] = 'https://fftoday.com/stats/playerstats.php?Season=' + temp_df['year'].astype(str) + '&GameWeek=' + temp_df['week'].astype(str) + '&PosID=99&LeagueID=26955'
    temp_df['page'] = temp_df['page'].astype(str)
    temp_df = temp_df.reset_index()
    temp_df = temp_df.drop(columns = ['index'])
    df2 = pd.DataFrame()

    for j in range(len(temp_df)):
        url = temp_df['page'][j]
        data = requests.get(url).text
        soup = BeautifulSoup(data, 'html.parser')
        dfs = pd.read_html(data)
        df = dfs[7]
        df = df.drop(labels = [0,1], axis = 0)
        df.columns = ['Team','G','Sack','FR', 'INT',
                   'DefTD','PA','PaYd/G','RuYd/G',
                 'Safety','KickTD','FPts','FPts/G']
        df['week_num'] = temp_df['week'][j]
        df['year'] = temp_df['year'][j]
        df2 = df2.append(df)
    
    new = df2['Team'].str.split(" ", expand = True)
    df2['Team'] = new[1]
    
    df2['abbr'] = df2['Team']
    df2['abbr'] = df2['abbr'].map({'Cardinals':'ARI','49ers':'SF','Saints':'NO',
                                   'Panthers':'CAR','Cowboys':'DAL','Texans':'HOU',
                                   'Raiders':'LV','Rams':'LAR','Lions':'DET',
                                   'Chiefs':'KC','Commanders':'WSH','Ravens':'BAL',
                                   'Colts':'IND','Steelers':'PIT','Bengals':'CIN',
                                   'Seahawks':'SEA','Dolphins':'MIA','Vikings':'MIN',
                                   'Patriots':'NE','Giants':'NYG','Titans':'TEN',
                                   'Broncos':'DEN', 'Jets':'NYJ','Eagles':'PHI',
                                   'Chargers':'LAC','Buccaneers':'TB','Bills':'BUF',
                                   'Browns':'CLE','Falcons':'ATL','Jaguars':'JAX',
                                   'Bears':'CHI','Packers':'GB'})
    
    df2 = df2.rename(columns = {'abbr':'CITY',
                               'PaYd/G':'PaYD',
                               'RuYd/G':'RuYd'})
    df2['Week'] = df2['week_num']
    df2['team_wk_year'] = df2['CITY'] + " " + df2['week_num'].astype(str) + " " + df2['year'].astype(str)
    nfl_sched['team_wk_year'] = nfl_sched['TEAM'] + " " + nfl_sched['Week'].astype(str) + " " + nfl_sched['Year'].astype(str)
    nfl_sched = nfl_sched.rename(columns = {'Opponent':'Opp'})
    nfl_sched = nfl_sched[['team_wk_year', 'Opp']]
    nfl_sched['away flag'] = np.where(nfl_sched['Opp'].astype(str).str[0] == '@', 1, 0)
    
    df2 = pd.merge(df2, nfl_sched, how = 'inner', on = 'team_wk_year', 
                  suffixes = (None, None))
    
    df2 = df2[['Team','CITY','Week','Sack','FR','INT','DefTD',
                'PA','PaYD','RuYd','Safety',
                'KickTD','FPts','week_num','year',
                'Opp','away flag','team_wk_year']]
    return df2
    

In [5]:
##now that we have the links df to grab player data and the 
##defense stats df from above, we can now create our qb modeling df

In [26]:
def position_feat_eng_df_creation(player_df,
                                  def_df,
                                  position,
                                 file_save):
    
    #if (position == 'QB'):
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(5)
            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df = dfs[0]
            df['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb = pd.concat([df_qb, df], axis=0, ignore_index=True)
        
        df_qb.columns = df_qb.columns.get_level_values(0) + '' +  df_qb.columns.get_level_values(1)
        if (position == 'QB'):
            print(df_qb.columns)

            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                       'Kick ReturnsRt','Kick ReturnsTD',
                                       'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       'Punt ReturnsRet', 'Punt ReturnsTD',
                                       'Punt ReturnsY/R','Punt ReturnsYds',
                                       'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                       'PuntingYds', 'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL','Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                       'Unnamed: 35_level_0Sk',# 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status','Unnamed: 53_level_0Status', 'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status'])
            print(df_qb.columns)
        
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds', 'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                        'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                        'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                        'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                        'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                        'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                        'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb.columns)

        elif (position == 'RB'):
            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       #'Punt ReturnsRet', 'Punt ReturnsTD',
                                       #'Punt ReturnsY/R','Punt ReturnsYds',
                                      #'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                      #'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                     #  'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk',# 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', #'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status',
                                             'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status'])
            print(df_qb.columns)
            df_qb = df_qb.drop(columns = ['Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
               'Def InterceptionsYds',  'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
               'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk','Unnamed: 19_level_0Sk',
               'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
               'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
               'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
               'Unnamed: 25_level_0Status', 'Unnamed: 27_level_0Status',
                'Unnamed: 30_level_0Sk','Unnamed: 30_level_0Status', 'Unnamed: 31_level_0Status',
               'Unnamed: 32_level_0Sk', 'Unnamed: 33_level_0Status',
               'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
               'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status',
               'Unnamed: 40_level_0Sk',
               'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
               'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
               'Unnamed: 50_level_0Status',
               'Unnamed: 52_level_0Status', 'Unnamed: 55_level_0Status',
               'Unnamed: 56_level_0Status', 'Unnamed: 57_level_0Status',
               'Unnamed: 58_level_0Status',
               'Unnamed: 62_level_0Status','Unnamed: 53_level_0Status','Unnamed: 35_level_0Sk'])
            print(df_qb.columns)

            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb.columns)
        
        elif (position == 'WR'):
            print(df_qb.columns)
            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct','Def InterceptionsInt',
                                           'Def InterceptionsPD', 'Def InterceptionsTD',
                                   'Def InterceptionsYds',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                    #   'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                     #  'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                      # 'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', #'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                           'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status','Unnamed: 59_level_0Status', 'Unnamed: 60_level_0Status',
                               'Unnamed: 62_level_0Status', 'Unnamed: 65_level_0Status',
                               'Unnamed: 66_level_0Status','ScoringXP%', 'ScoringXPA', 'ScoringXPM',
                                           'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
                                   'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk','Unnamed: 19_level_0Sk',
                                          'Unnamed: 20_level_0Sk', 'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                   'Unnamed: 23_level_0Sk', 'Unnamed: 23_level_0Status',
                                   'Unnamed: 24_level_0Sk', 'Unnamed: 24_level_0Status',
                                   'Unnamed: 25_level_0Sk', 'Unnamed: 25_level_0Status',
                                       'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                           'Unnamed: 29_level_0Status','Unnamed: 30_level_0Sk',
                                   'Unnamed: 30_level_0Status', 'Unnamed: 31_level_0Sk',
                                   'Unnamed: 31_level_0Status', 'Unnamed: 32_level_0Sk',
                                   'Unnamed: 33_level_0Status', 'Unnamed: 35_level_0Status',
                                           'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
                                       'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status','Unnamed: 40_level_0Sk', 'Unnamed: 41_level_0Sk',
                                   'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                                   'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
                                   'Unnamed: 49_level_0Status','Unnamed: 50_level_0Status',
                                   'Unnamed: 52_level_0Status', 'Unnamed: 55_level_0Status',
                                   'Unnamed: 56_level_0Status', 'Unnamed: 57_level_0Status',
                                   'Unnamed: 58_level_0Status','ScoringFG%', 'ScoringFGA', 'ScoringFGM'
                                          ])


            print(df_qb.columns)
        
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb.columns)
         
        elif (position == 'TE'):
            print(df_qb.columns)
            df_qb = df_qb.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                            'Def InterceptionsInt', 
                                           'Def InterceptionsPD', 'Def InterceptionsTD',
                                           'Def InterceptionsYds', 
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                    #   'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                     #  'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                      # 'Unnamed: 26_level_0Sk', 
                                           'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                      # 'Unnamed: 33_level_0Sk', 
                                           'Unnamed: 34_level_0Status',
                                      # 'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', #'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status',#'Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                           'Unnamed: 54_level_0Status',
                                            'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
                                           'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk',
                                           'Unnamed: 19_level_0Sk','Unnamed: 20_level_0Sk',
                                           'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                           'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
                                           'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                                           'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                           'Unnamed: 29_level_0Status', 'Unnamed: 30_level_0Status',
                                           'Unnamed: 31_level_0Status', 'Unnamed: 32_level_0Sk',
                                           'Unnamed: 33_level_0Status', 'Unnamed: 35_level_0Sk',
                                           'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Sk',
                                           'Unnamed: 37_level_0Status', 'Unnamed: 39_level_0Status',
                                           'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                                           'Unnamed: 45_level_0Status',
                                            'Unnamed: 50_level_0Status',
                                           'Unnamed: 55_level_0Status', 'Unnamed: 58_level_0Status'
                               # 'Unnamed: 61_level_0Status'
                                          ])

            print(df_qb.columns)
        
            df_qb = df_qb.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
         
            print(df_qb.columns)
        
    
        
        
        
        df_qb2 = df_qb[df_qb['G#'] != '']
        df_qb2 = df_qb2[df_qb2['G#'] != 'G#']
        df_qb2 = df_qb2[~df_qb2['Date'].str.contains("Game")]
        df_qb2 = df_qb2[df_qb2['GS'] == '*']
        df_qb2['player_wk_year'] = df_qb2['Player '] + " " + df_qb2['Week'].astype(str) + " " + df_qb2['Year'].astype(str)
        print('yay')
        
        
        test_df = df_qb2[['Player ','Week', 'Year', 'player_wk_year']]
        test_df['Week'] = test_df['Week'].astype(int)
        test_df['Year'] = test_df['Year'].astype(int)        
        for i in range(1,36):
            print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            player_wks_ago = "player " + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "Week"
                curr_yr = "Year"
                test_df[curr_wk] = test_df['Week']
                test_df[curr_yr] = test_df['Year']
               # test_df[curr_wk] = test_df[curr_wk].astype(int)
               # test_df[curr_yr] = test_df[curr_yr].astype(int)
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])

            test_df[player_wks_ago] = test_df['Player '] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

            
       # test_df.head()
        new_df = pd.DataFrame()
        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df2 = test_df.drop(columns = ['Player ', 'Week', 'Year'])
                new_df = pd.merge(test_df2, df_qb2, how = 'left', on = ['player_wk_year'], suffixes = (None, None))
                print(new_df.head())
            else:
                temp_df = df_qb2.drop(columns = ['Rk','GS','Year','G#'])
                for col in temp_df.columns:
                    new_col = col + " " + str(i) + " wks ago"
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                merge_column = "player " + str(i) + " wks ago"
                temp_df[merge_column] = df_qb2['player_wk_year']
                new_df = pd.merge(new_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
        
        qb_df = new_df

        tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
              'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
              'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
              'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
              'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
              'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
              'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
              'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
              'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
              'Tm 35 wks ago']

        opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
              'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
              'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
              'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
              'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
              'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
              'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
              'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
              'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
              'Opp 35 wks ago']
        
        for i in tms:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        for i in opp:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE', 'NOR': 'NO',
                              'SFO': 'SF', 'SDG':'LAC', 'GNB':'GB','TAM':'TB',
                              'WAS':'WSH','KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV','IND':'IND', 'SEA':'SEA', 'HOU':'HOU',
                             'ATL':'ATL','CIN':'CIN', 'PIT':'PIT', 'NYG':'NYG',
                             'DET':'DET','CLE':'CLE', 'DEN':'DEN', 'DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI','BAL':'BAL',
                             'JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA', 'CAR':'CAR',
                             'PHI':'PHI', 'MIN':'MIN', 'TEN':'TEN', 'LAR':'LAR',
                             'LAC':'LAC'})    
            
        #opp
        opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

        wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

        year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
        
        for i in range(0,36):
            #print(i)
            #print(opp[i])
            #print(wk[i])
            #print(year[i])
            opp_colname = opp[i]
            wk_colname = wk[i]
            year_colname = year[i]
            opp_wk_year_colname = opp_wk_year[i]
            if (i== 0):
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
            else:
                #qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str).str[:-2] + " " + qb_df[year_colname].astype(str)
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
        
        #if (i == 0):

        test_df = qb_df[['player_wk_year','opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']]

        new_df = pd.DataFrame()

        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                #test_df2 = test_df.drop(columns = [])
                new_df = pd.merge(test_df, def_df, how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                #print(new_df.head())
            else:
                temp_df = def_df.drop(columns = ['Week','week_num','year','Opp','team_wk_year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'opp' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                temp_df['team_wk_year'] = def_df['team_wk_year']
               # print(temp_df2.head())
                qb_no_null = test_df[[opp_wk_year[i]]]
               # print(qb_no_null.head())
                qb_no_null = qb_no_null[qb_no_null.notnull()]
              #  print(qb_no_null.head())
                qb_no_null = pd.merge(qb_no_null, temp_df, how = 'inner', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None,None))
                #print(qb_no_null.head())
                qb_no_null = qb_no_null.drop_duplicates()
        
                qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                new_df = pd.merge(new_df, qb_no_null, how = 'left', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
        
        new_df = new_df.drop_duplicates()
        
        qb_df2 = qb_df.drop(columns = ['Week', 'Opp', 'opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago',
           'opp 3 wk year ago', 'opp 4 wk year ago', 'opp 5 wk year ago',
           'opp 6 wk year ago', 'opp 7 wk year ago', 'opp 8 wk year ago',
           'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
           'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago',
           'opp 15 wk year ago', 'opp 16 wk year ago', 'opp 17 wk year ago',
           'opp 18 wk year ago', 'opp 19 wk year ago', 'opp 20 wk year ago',
           'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
           'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago',
           'opp 27 wk year ago', 'opp 28 wk year ago', 'opp 29 wk year ago',
           'opp 30 wk year ago', 'opp 31 wk year ago', 'opp 32 wk year ago',
           'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago'])
        

        new_df2 = pd.merge(qb_df2,new_df, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        
        test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            #player_wks_ago = "player " + str(i) + " wks ago"
            team_wk_year = 'team_wk_year ' + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "week_num"
                curr_yr = "year"
                test_df[curr_wk] = test_df['week_num']
                test_df[curr_yr] = test_df['year']
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])

            test_df[team_wk_year] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

        #test_df.head()
        new_def_df = pd.DataFrame()

        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df = test_df.drop(columns = ['CITY','week_num','year'])
                new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
              #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
                print(new_def_df.head())
            else:
                temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                merge_column = "team_wk_year " + str(i) + " wks ago"
                temp_df[merge_column] = def_df['team_wk_year']
                #temp_df['team_wk_year'] = def_df['team_wk_year']
               # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
        new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
               'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
               'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
               'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
               'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
               'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
               'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
               'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
               'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
               'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
               'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
               'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
               'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
               'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
               'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
               'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
               'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
               'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
               'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
               'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
               'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
               'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
               'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
               'year 35 wks ago', 'Team', 'CITY', 'Week', 'Sack', 'FR', 'INT', 'DefTD',
               'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD', 'FPts', 'week_num',
               'year', 'Opp', 'away flag', 'opp_wk_year'
                                                ])
        
        new_df3 = pd.merge(new_df2,new_def_df2,how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None,None))
        
        new_df3.to_csv(file_save)
        return new_df3

In [27]:
def create_qb_trend_var(qb_df, position, file_save):
    qb_df = qb_df.drop(columns = [#'Unnamed: 0',
                                  'week num 1 wks ago',
        'year 1 wks ago','player 1 wks ago','week num 2 wks ago','year 2 wks ago',
        'player 2 wks ago','week num 3 wks ago','year 3 wks ago','player 3 wks ago',
        'week num 4 wks ago','year 4 wks ago','player 4 wks ago','week num 5 wks ago',
        'year 5 wks ago','player 5 wks ago','week num 6 wks ago','year 6 wks ago',
        'player 6 wks ago','week num 7 wks ago','year 7 wks ago','player 7 wks ago',
        'week num 8 wks ago','year 8 wks ago','player 8 wks ago','week num 9 wks ago',
        'year 9 wks ago','player 9 wks ago','week num 10 wks ago','year 10 wks ago',
        'player 10 wks ago','week num 11 wks ago','year 11 wks ago','player 11 wks ago',
        'week num 12 wks ago','year 12 wks ago','player 12 wks ago','week num 13 wks ago',
        'year 13 wks ago','player 13 wks ago','week num 14 wks ago','year 14 wks ago',
        'player 14 wks ago','week num 15 wks ago','year 15 wks ago','player 15 wks ago',
        'week num 16 wks ago','year 16 wks ago','player 16 wks ago','week num 17 wks ago',
        'year 17 wks ago','player 17 wks ago','week num 18 wks ago','year 18 wks ago',
        'player 18 wks ago','week num 19 wks ago','year 19 wks ago','player 19 wks ago',
        'week num 20 wks ago','year 20 wks ago','player 20 wks ago','week num 21 wks ago',
        'year 21 wks ago','player 21 wks ago','week num 22 wks ago','year 22 wks ago',
        'player 22 wks ago','week num 23 wks ago','year 23 wks ago','player 23 wks ago',
        'week num 24 wks ago','year 24 wks ago','player 24 wks ago','week num 25 wks ago',
        'year 25 wks ago','player 25 wks ago','week num 26 wks ago','year 26 wks ago',
        'player 26 wks ago','week num 27 wks ago','year 27 wks ago','player 27 wks ago',
        'week num 28 wks ago','year 28 wks ago','player 28 wks ago','week num 29 wks ago',
        'year 29 wks ago','player 29 wks ago','week num 30 wks ago','year 30 wks ago',
        'player 30 wks ago','week num 31 wks ago','year 31 wks ago','player 31 wks ago',
        'week num 32 wks ago','year 32 wks ago','player 32 wks ago','week num 33 wks ago',
        'year 33 wks ago','player 33 wks ago','week num 34 wks ago','year 34 wks ago',
        'player 34 wks ago','week num 35 wks ago','year 35 wks ago','player 35 wks ago',
        #'Unnamed: 0.1','Unnamed: 0.1.1',
        'opp 1 wk year ago','opp 2 wk year ago',
        'opp 3 wk year ago','opp 4 wk year ago','opp 5 wk year ago','opp 6 wk year ago',
        'opp 7 wk year ago','opp 8 wk year ago','opp 9 wk year ago','opp 10 wk year ago',
        'opp 11 wk year ago','opp 12 wk year ago','opp 13 wk year ago','opp 14 wk year ago',
        'opp 15 wk year ago','opp 16 wk year ago','opp 17 wk year ago','opp 18 wk year ago',
        'opp 19 wk year ago','opp 20 wk year ago','opp 21 wk year ago','opp 22 wk year ago',
        'opp 23 wk year ago','opp 24 wk year ago','opp 25 wk year ago','opp 26 wk year ago',
        'opp 27 wk year ago','opp 28 wk year ago','opp 29 wk year ago','opp 30 wk year ago',
        'opp 31 wk year ago','opp 32 wk year ago','opp 33 wk year ago','opp 34 wk year ago',
        'opp 35 wk year ago','team_wk_year 1 wks ago','team_wk_year 2 wks ago','team_wk_year 3 wks ago',
        'team_wk_year 4 wks ago','team_wk_year 5 wks ago','team_wk_year 6 wks ago','team_wk_year 7 wks ago',
        'team_wk_year 8 wks ago','team_wk_year 9 wks ago','team_wk_year 10 wks ago','team_wk_year 11 wks ago',
        'team_wk_year 12 wks ago','team_wk_year 13 wks ago','team_wk_year 14 wks ago','team_wk_year 15 wks ago',
        'team_wk_year 16 wks ago','team_wk_year 17 wks ago','team_wk_year 18 wks ago','team_wk_year 19 wks ago',
        'team_wk_year 20 wks ago','team_wk_year 21 wks ago','team_wk_year 22 wks ago','team_wk_year 23 wks ago',
        'team_wk_year 24 wks ago','team_wk_year 25 wks ago','team_wk_year 26 wks ago','team_wk_year 27 wks ago',
        'team_wk_year 28 wks ago','team_wk_year 29 wks ago','team_wk_year 30 wks ago','team_wk_year 31 wks ago',
        'team_wk_year 32 wks ago','team_wk_year 33 wks ago','team_wk_year 34 wks ago','team_wk_year 35 wks ago'
        ])
    qb_df = qb_df.drop_duplicates()
    print(qb_df.shape)
    
    print('trend 1 creation')
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = qb_df[['player_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['opp Sack', 'opp FR', 'opp INT', 'opp DefTD', 'opp PA', 'opp PaYD', 'opp RuYd',
       'opp Safety', 'opp KickTD', 'opp FPts', 'opp away flag']
    
    #for feat in features:
    #    qb_df[feat] = qb_df[feat].astype('float')
    
    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            qb_df[fillna_col] = qb_df[fillna_col].fillna(0)
            qb_df[fillna_col] = qb_df[fillna_col].astype('float')
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = qb_df[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     
    print('merge 1')
    qb_df = pd.merge(qb_df, temp_df, how = 'inner', on= 'player_wk_year', suffixes = (None,None))
    qb_df = qb_df.drop_duplicates()
    print(qb_df.shape)
    
    print('trend 2 creation')

    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = qb_df[['player_wk_year']]    
    temp_df2 = pd.DataFrame()     
    if (position == 'QB'):
        
        features = ['Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
        'Fumbles TD','Fumbles Yds','Off. Snaps Num','Off. Snaps Pct','Passing AY/A',
        'Passing Att','Passing Cmp','Passing Cmp%','Passing Int','Passing Rate',
        'Passing Sk','Passing TD','Passing Y/A','Passing Yds',
        'Rushing Att','Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
        'ST Snaps Pct','Scoring 2PM','Scoring Pts','Scoring TD',
        'Age']
    elif(position == 'RB' or position == 'WR' or
        position == 'TE'):
        
        features = ['Fumbles FF','Fumbles FL','Fumbles FR','Fumbles Fmb',
        'Fumbles TD','Fumbles Yds','Kick Returns Rt','Kick Returns TD',
        'Kick Returns Y/Rt','Kick Returns Yds','Off. Snaps Num','Off. Snaps Pct',
        'Punt Returns Ret','Punt Returns TD','Punt Returns Y/R','Punt Returns Yds',
        'Receiving Ctch%','Receiving Rec','Receiving TD','Receiving Tgt',
        'Receiving Y/R','Receiving Y/Tgt','Receiving Yds','Rushing Att',
        'Rushing TD','Rushing Y/A','Rushing Yds','ST Snaps Num',
        'ST Snaps Pct','Scoring 2PM','Scoring Pts',#'Scoring Sfty',
        'Scoring TD','Age']
        
    #for feat in features:
    #    qb_df[feat] = qb_df[feat].astype('float')

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + " wks ago"
            qb_df[fillna_col] = qb_df[fillna_col].fillna(0)
            if (feat == 'Off. Snaps Pct' or feat == 'Receiving Ctch%' or
               feat == 'ST Snaps Pct' or 'Passing Cmp%'):
                qb_df[fillna_col] = qb_df[fillna_col].astype(str)
                qb_df[fillna_col] = qb_df[fillna_col].str.rstrip('%').astype('float')/100.0
            else:   
                #print('not feat')
                qb_df[fillna_col] = qb_df[fillna_col].astype('float')
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = qb_df[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     

    print('merge 2')
    qb_df = pd.merge(qb_df, temp_df, how = 'inner', on= 'player_wk_year', suffixes = (None,None))
    qb_df = qb_df.drop_duplicates()

    print(qb_df.shape)

    print('trend 3 creation')
    
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = qb_df[['player_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['def matchup Sack','def matchup FR',
    'def matchup INT','def matchup DefTD',
    'def matchup PA','def matchup PaYD',
    'def matchup RuYd','def matchup Safety',
    'def matchup KickTD','def matchup FPts',
    #'def matchup Opp',
                'def matchup away flag']
    
   # for feat in features:
   #     qb_df[feat] = qb_df[feat].astype('float')

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            qb_df[fillna_col] = qb_df[fillna_col].fillna(0)
            qb_df[fillna_col] = qb_df[fillna_col].astype('float')
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = qb_df[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     

    print('merge 3')
    qb_df = pd.merge(qb_df, temp_df, how = 'inner', on= 'player_wk_year', suffixes = (None,None))
    qb_df = qb_df.drop_duplicates()

    print(qb_df.shape)

    qb_df.to_csv(file_save)
    return qb_df

In [28]:
#now we need to get our current week players and get an oot df
#for the week we care about
def create_curr_wk_oot_qb(nfl_sched,
                          year,
                          week,
                          player_df,
                          position,
                          def_df,
                         save_file):
    
        nfl_sched = nfl_sched[(nfl_sched['Year'] == year) &
                             (nfl_sched['Week'] == week)]
        nfl_sched = nfl_sched.reset_index()
        nfl_sched = nfl_sched.drop(columns = ['index'])
    
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        qb = qb[qb['Player'] != 'Brandon Allen']

        qb = qb[qb['Playing_Prd_End'] ==  year]
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(5)
            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df = dfs[0]
            df['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb = pd.concat([df_qb, df], axis=0, ignore_index=True)
        
        df_qb.columns = df_qb.columns.get_level_values(0) + '' +  df_qb.columns.get_level_values(1)
        df_qb = df_qb[['Player','Unnamed: 6_level_0Tm']]
        df_qb = df_qb.rename(columns = {'Unnamed: 6_level_0Tm':'TEAM'})
        df_qb = df_qb.drop_duplicates()
        df_qb['TEAM'] = df_qb['TEAM'].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        
        df_qb = pd.merge(df_qb, nfl_sched, how = 'inner', on = 'TEAM', 
                        suffixes = (None, None))
        df_qb = df_qb.rename(columns = {'TEAM':'team',
                                       'Player':'Player ',
                                       'Opponent':'opp'})
        df_qb['opp'] = df_qb['opp'].str.replace('@',"")

        df_qb['player_wk_year'] = df_qb['Player '] + " " + df_qb['Week'].astype(str) + " " + df_qb['Year'].astype(str)
        df_qb['opp wk year'] = df_qb['opp'] + " " + df_qb['Week'].astype(str) + " " + df_qb['Year'].astype(str)
        
        df = df_qb[['Week','Year','Player ','team',
                      'opp','player_wk_year','opp wk year']]
        
        
        qb = player_df[player_df['Position'] == position]
        qb = qb[qb['Player'] != 'Malik Willis']
        qb = qb.reset_index()
        qb = qb.drop(columns = ['index'])
        
        df_qb2 = pd.DataFrame()
        #for i in range(1000):
        for i in range(len(qb)):
            time.sleep(5)

            print(i)
            player = qb['Player'][i]
            print(qb['Player'][i])
            url = qb['Gamelog'][i]
            #print(qb['Player'][i])
            data = requests.get(url).text
            #print(data)
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup)
            dfs = pd.read_html(data)
            df2 = dfs[0]
            df2['Player'] = player
            #print(df.head())    
            #print(df.columns)
            df_qb2 = pd.concat([df_qb2, df2], axis=0, ignore_index=True)
        
        df_qb2.columns = df_qb2.columns.get_level_values(0) + '' +  df_qb2.columns.get_level_values(1)
        if (position == 'QB'):
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                       'Kick ReturnsRt','Kick ReturnsTD',
                                       'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       'Punt ReturnsRet', 'Punt ReturnsTD',
                                       'Punt ReturnsY/R','Punt ReturnsYds',
                                       'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                       'PuntingYds', 'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL','Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                       'Unnamed: 35_level_0Sk', #'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status','Unnamed: 53_level_0Status', 'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status'])
            print(df_qb2.columns)
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds', 'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                        'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                        'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                        'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                        'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                        'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                        'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
            print(df_qb2.columns)
        elif (position == 'WR'):
            

            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                       #'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                       #'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                       #'Unnamed: 35_level_0Sk', 
                                             'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk',# 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                             'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status','Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
                           'Def InterceptionsYds', 'ScoringFG%', 'ScoringFGA', 'ScoringFGM', 
                                              'ScoringXP%', 'ScoringXPA',
                               'ScoringXPM', 'Unnamed: 11_level_0Sk',
                               'Unnamed: 15_level_0Sk', 'Unnamed: 17_level_0Status',
                               'Unnamed: 18_level_0Sk', 'Unnamed: 19_level_0Sk',
                                              'Unnamed: 20_level_0Sk', 'Unnamed: 21_level_0Sk',
                               'Unnamed: 22_level_0Sk', 'Unnamed: 23_level_0Sk',
                               'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
                               'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                               'Unnamed: 25_level_0Status', 'Unnamed: 26_level_0Status',
                               'Unnamed: 27_level_0Status', 'Unnamed: 29_level_0Status', 
                                             'Unnamed: 30_level_0Sk', 'Unnamed: 30_level_0Status',
                               'Unnamed: 31_level_0Sk', 'Unnamed: 31_level_0Status',
                               'Unnamed: 32_level_0Sk', 'Unnamed: 33_level_0Status',
                               'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
                               'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status',
                                             'Unnamed: 40_level_0Sk', 'Unnamed: 41_level_0Sk',
                               'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                               'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
                                 'Unnamed: 50_level_0Status', 'Unnamed: 52_level_0Status',
                               'Unnamed: 55_level_0Status', 'Unnamed: 56_level_0Status',
                               'Unnamed: 57_level_0Status', 'Unnamed: 58_level_0Status',
                               'Unnamed: 59_level_0Status',   'Unnamed: 60_level_0Status',
                               'Unnamed: 62_level_0Status', 'Unnamed: 65_level_0Status',
                               'Unnamed: 66_level_0Status'])
        

            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
                
        elif (position == 'TE'):
            
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                      # 'Punt ReturnsRet', 'Punt ReturnsTD',
                                      # 'Punt ReturnsY/R','Punt ReturnsYds',
                                      # 'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                     #  'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                      # 'Unnamed: 26_level_0Sk', 
                                             'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                      # 'Unnamed: 33_level_0Sk', 
                                             'Unnamed: 34_level_0Status',
                                       'Unnamed: 35_level_0Sk',# 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk', 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status',#'Unnamed: 43_level_0Sk',
                                             #'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status',# 'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status', 
                                             'Unnamed: 54_level_0Status',
                                #'Unnamed: 61_level_0Status'
                                             'Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
                                           'Def InterceptionsYds',  'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
                                           'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk',
                                       'Unnamed: 19_level_0Sk','Unnamed: 20_level_0Sk',
                                       'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
                                       'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
                                       'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
                                       'Unnamed: 26_level_0Status', 'Unnamed: 27_level_0Status',
                                       'Unnamed: 29_level_0Status','Unnamed: 30_level_0Status',
                                       'Unnamed: 31_level_0Status', 'Unnamed: 32_level_0Sk',
                                           'Unnamed: 33_level_0Status', 'Unnamed: 36_level_0Status',
                                       'Unnamed: 37_level_0Status', 'Unnamed: 39_level_0Status',
                                             'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
                                       'Unnamed: 45_level_0Status','Unnamed: 50_level_0Status',
                                       'Unnamed: 55_level_0Status', 'Unnamed: 58_level_0Status'
                                            ])


            print(df_qb2.columns)
            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
                
        
        elif (position == 'RB'):
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns  = ['Def. SnapsNum', 'Def. SnapsPct',
                                      # 'Kick ReturnsRt','Kick ReturnsTD',
                                      # 'Kick ReturnsY/Rt', 'Kick ReturnsYds',
                                       #'Punt ReturnsRet', 'Punt ReturnsTD',
                                       #'Punt ReturnsY/R','Punt ReturnsYds',
                                      #'PuntingBlck', 'PuntingPnt', 'PuntingY/P',
                                      #'PuntingYds', 
                                        #'ReceivingCtch%', 'ReceivingRec', 'ReceivingTD',
                                       #'ReceivingTgt', 'ReceivingY/R', 'ReceivingY/Tgt', 'ReceivingYds',
                                       'TacklesAst', 'TacklesComb',
                                       'TacklesQBHits', 'TacklesSolo',
                                       'TacklesTFL',
                                             'PassingAY/A','PassingAtt',
                                        'PassingCmp', 'PassingCmp%',
                                        'PassingInt', 'PassingRate',
                                        'PassingSk', 'PassingTD',
                                        'PassingY/A', 'PassingYds',
                                        'PassingYds.1',
                                           'Unnamed: 21_level_0Status',
                                       'Unnamed: 26_level_0Sk', 'Unnamed: 28_level_0Sk',
                                       'Unnamed: 28_level_0Status', 'Unnamed: 29_level_0Sk',
                                       'Unnamed: 32_level_0Status',
                                       'Unnamed: 33_level_0Sk', 'Unnamed: 34_level_0Status',
                                     #  'Unnamed: 35_level_0Sk', 'Unnamed: 35_level_0Status',
                                       'Unnamed: 36_level_0Sk',# 'Unnamed: 37_level_0Sk',
                                       'Unnamed: 38_level_0Status',  'Unnamed: 40_level_0Status',
                                       'Unnamed: 41_level_0Status','Unnamed: 43_level_0Sk', 'Unnamed: 44_level_0Status',
                                       'Unnamed: 46_level_0Status', 'Unnamed: 47_level_0Status',
                                       'Unnamed: 48_level_0Status', #'Unnamed: 49_level_0Status',
                                'Unnamed: 51_level_0Status',#'Unnamed: 53_level_0Status',
                                             'Unnamed: 54_level_0Status',
                                'Unnamed: 61_level_0Status',
                                            'Unnamed: 35_level_0Sk', 'Unnamed: 53_level_0Status'])
            
            print(df_qb2.columns)
            df_qb2 = df_qb2.drop(columns = ['Def InterceptionsInt', 'Def InterceptionsPD', 'Def InterceptionsTD',
               'Def InterceptionsYds',  'Unnamed: 11_level_0Sk', 'Unnamed: 15_level_0Sk',
               'Unnamed: 17_level_0Status', 'Unnamed: 18_level_0Sk','Unnamed: 19_level_0Sk',
               'Unnamed: 21_level_0Sk', 'Unnamed: 22_level_0Sk',
               'Unnamed: 23_level_0Status', 'Unnamed: 24_level_0Sk',
               'Unnamed: 24_level_0Status', 'Unnamed: 25_level_0Sk',
               'Unnamed: 25_level_0Status', 'Unnamed: 27_level_0Status',
                'Unnamed: 30_level_0Sk','Unnamed: 30_level_0Status', 'Unnamed: 31_level_0Status',
               'Unnamed: 32_level_0Sk', 'Unnamed: 33_level_0Status',
               'Unnamed: 36_level_0Status', 'Unnamed: 37_level_0Status',
               'Unnamed: 39_level_0Sk', 'Unnamed: 39_level_0Status',
               'Unnamed: 40_level_0Sk',
               'Unnamed: 42_level_0Status', 'Unnamed: 43_level_0Status',
               'Unnamed: 44_level_0Sk', 'Unnamed: 45_level_0Status',
               'Unnamed: 50_level_0Status',
               'Unnamed: 52_level_0Status', 'Unnamed: 55_level_0Status',
               'Unnamed: 56_level_0Status', 'Unnamed: 57_level_0Status',
               'Unnamed: 58_level_0Status',
               'Unnamed: 62_level_0Status'])
            print(df_qb2.columns)

            df_qb2 = df_qb2.rename(columns = {'FumblesFF':'Fumbles FF',
                                  'FumblesFL':'Fumbles FL',
                                  'FumblesFR':'Fumbles FR',
                                  'FumblesFmb':'Fumbles Fmb',
                                  'FumblesTD':'Fumbles TD',
                                        'FumblesYds':'Fumbles Yds',
                                               'Kick ReturnsRt':'Kick Returns Rt',
                                            'Kick ReturnsTD':'Kick Returns TD',
                                       'Kick ReturnsY/Rt':'Kick Returns Y/Rt',
                                            'Kick ReturnsYds':'Kick Returns Yds',
                                            'Off. SnapsNum':'Off. Snaps Num',
                                        'Off. SnapsPct':'Off. Snaps Pct',
                                       # 'PassingAY/A':'Passing AY/A','PassingAtt':'Passing Att',
                                       # 'PassingCmp':'Passing Cmp', 'PassingCmp%':'Passing Cmp%',
                                       # 'PassingInt':'Passing Int', 'PassingRate':'Passing Rate',
                                       # 'PassingSk':'Passing Sk', 'PassingTD': 'Passing TD',
                                       # 'PassingY/A':'Passing Y/A', 'PassingYds':'Passing Yds',
                                       # 'PassingYds.1':'Passing Yds.1',
                                        'Player':'Player ', 
                                            'Punt ReturnsRet':'Punt Returns Ret',
                                            'Punt ReturnsTD':'Punt Returns TD',
                                       'Punt ReturnsY/R':'Punt Returns Y/R',
                                            'Punt ReturnsYds':'Punt Returns Yds',
                                               
                                            'ReceivingCtch%':'Receiving Ctch%',
                                            'ReceivingRec':'Receiving Rec',
                                            'ReceivingTD':'Receiving TD',
                                       'ReceivingTgt':'Receiving Tgt',
                                            'ReceivingY/R':'Receiving Y/R',
                                            'ReceivingY/Tgt':'Receiving Y/Tgt',
                                            'ReceivingYds':'Receiving Yds',
                                            
                                            'RushingAtt':'Rushing Att',
                                        'RushingTD':'Rushing TD', 'RushingY/A':'Rushing Y/A',
                                        'RushingYds':'Rushing Yds',
                                        'ST SnapsNum':'ST Snaps Num', 'ST SnapsPct':'ST Snaps Pct',
                                        'Scoring2PM':'Scoring 2PM', 'ScoringPts':'Scoring Pts',
                                        'ScoringTD':'Scoring TD',
                                            
                                     #   'TacklesAst':'Tackles Ast',
                                      #      'TacklesComb':'Tackles Comb',
                                      # 'TacklesQBHits':'Tackles QBHits',
                                      #      'TacklesSolo':'Tackles Solo',
                                      # 'TacklesTFL':'TacklesTFL',
                                            
                                        'Unnamed: 0_level_0Rk':'Rk', 'Unnamed: 10_level_0GS':'GS',
                                        'Unnamed: 1_level_0Year':'Year', 'Unnamed: 2_level_0Date':'Date',
                                        'Unnamed: 3_level_0G#':'G#', 'Unnamed: 4_level_0Week':'Week',
                                        'Unnamed: 5_level_0Age':'Age', 'Unnamed: 6_level_0Tm':'Tm',
                                        'Unnamed: 7_level_0Unnamed: 7_level_1':'home_away',
                                  'Unnamed: 8_level_0Opp':'Opp',
                                        'Unnamed: 9_level_0Result':'Result'})
        print(df_qb2.columns)
    
        df_qb2 = df_qb2[df_qb2['G#'] != '']
        df_qb2 = df_qb2[df_qb2['G#'] != 'G#']
        df_qb2 = df_qb2[~df_qb2['Date'].str.contains("Game")]
        df_qb2 = df_qb2[df_qb2['GS'] == '*']
        df_qb2['player_wk_year'] = df_qb2['Player '] + " " + df_qb2['Week'].astype(str) + " " + df_qb2['Year'].astype(str)
        print('yay')
        
        test_df = df[['Player ','Week', 'Year', 'player_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            player_wks_ago = "player " + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "Week"
                curr_yr = "Year"
                test_df[curr_wk] = test_df['Week']
                test_df[curr_yr] = test_df['Year']
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            test_df[player_wks_ago] = test_df['Player '] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)
    
        new_df = pd.DataFrame()
        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df2 = test_df.drop(columns = ['Player ', 'Week', 'Year'])
                new_df = pd.merge(test_df2, df_qb2, how = 'left', on = ['player_wk_year'], suffixes = (None, None))
                print(new_df.head())
            else:
                temp_df = df_qb2.drop(columns = ['Rk','GS','Year','G#'])
                for col in temp_df.columns:
                    new_col = col + " " + str(i) + " wks ago"
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                merge_column = "player " + str(i) + " wks ago"
                temp_df[merge_column] = df_qb2['player_wk_year']
                new_df = pd.merge(new_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                new_df = new_df.drop_duplicates()
        
        tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
          'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
          'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
          'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
          'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
          'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
          'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
          'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
          'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
          'Tm 35 wks ago']

        opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
          'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
          'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
          'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
          'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
          'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
          'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
          'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
          'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
          'Opp 35 wks ago']
        
        qb_df = new_df
        
        for i in tms:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE',
                               'NOR': 'NO', 'SFO': 'SF', 'SDG':'LAC',
                              'GNB':'GB','TAM':'TB','WAS':'WSH',
                              'KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV', 'IND':'IND', 'SEA':'SEA',
                             'HOU':'HOU', 'ATL':'ATL', 'CIN':'CIN',
                             'PIT':'PIT','NYG':'NYG', 'DET':'DET',
                             'CLE':'CLE','DEN':'DEN','DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI',
                             'BAL':'BAL','JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA',
                             'CAR':'CAR','PHI':'PHI','MIN':'MIN',
                             'TEN':'TEN', 'LAR':'LAR', 'LAC':'LAC'})
        for i in opp:
            #print(i)
            qb_df[i] = qb_df[i].map({'NWE': 'NE', 'NOR': 'NO',
                              'SFO': 'SF', 'SDG':'LAC', 'GNB':'GB','TAM':'TB',
                              'WAS':'WSH','KAN':'KC', 'OAK':'LV', 'STL':'LAR',
                              'LVR':'LV','IND':'IND', 'SEA':'SEA', 'HOU':'HOU',
                             'ATL':'ATL','CIN':'CIN', 'PIT':'PIT', 'NYG':'NYG',
                             'DET':'DET','CLE':'CLE', 'DEN':'DEN', 'DAL':'DAL',
                             'CHI':'CHI','BUF':'BUF','ARI':'ARI','BAL':'BAL',
                             'JAX':'JAX', 'NYJ':'NYJ','MIA':'MIA', 'CAR':'CAR',
                             'PHI':'PHI', 'MIN':'MIN', 'TEN':'TEN', 'LAR':'LAR',
                             'LAC':'LAC'})    
            
        #opp
        opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

        wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

        year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
        
        for i in range(0,36):
            print(i)
            #print(opp[i])
            #print(wk[i])
            #print(year[i])
            opp_colname = opp[i]
            wk_colname = wk[i]
            year_colname = year[i]
            opp_wk_year_colname = opp_wk_year[i]
            if (i== 0):
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)
            else:
               # qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str).str[:-2] + " " + qb_df[year_colname].astype(str)
                qb_df[opp_wk_year_colname] = qb_df[opp_colname] + " " + qb_df[wk_colname].astype(str) + " " + qb_df[year_colname].astype(str)

                #if (i == 0):

        test_df = qb_df[['player_wk_year','opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']]

        new_df = pd.DataFrame()

        for i in range(0,36):
            print(i)
            if (i == 0):
                #new_df = df_qb2
                #test_df2 = test_df.drop(columns = [])
                new_df = pd.merge(test_df, def_df, how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                #print(new_df.head())
            else:
                temp_df = def_df.drop(columns = ['Week','week_num','year','Opp','team_wk_year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'opp' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                temp_df['team_wk_year'] = def_df['team_wk_year']
               # print(temp_df2.head())
                qb_no_null = test_df[[opp_wk_year[i]]]
               # print(qb_no_null.head())
                qb_no_null = qb_no_null[qb_no_null.notnull()]
                qb_no_null = qb_no_null.drop_duplicates()
              #  print(qb_no_null.head())
                qb_no_null = pd.merge(qb_no_null, temp_df, how = 'inner', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None,None))
                #print(qb_no_null.head())
        
                qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                qb_no_null = qb_no_null.drop_duplicates()
                new_df = pd.merge(new_df, qb_no_null, how = 'left', left_on = [opp_wk_year[i]], right_on = ['team_wk_year'], suffixes = (None, None))
                new_df = new_df.drop(columns = ['team_wk_year'])
                new_df = new_df.drop_duplicates()
        
        new_df = new_df.drop_duplicates()
        
        qb_df2 = qb_df.drop(columns = ['Week', 'Opp', 'opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago',
           'opp 3 wk year ago', 'opp 4 wk year ago', 'opp 5 wk year ago',
           'opp 6 wk year ago', 'opp 7 wk year ago', 'opp 8 wk year ago',
           'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
           'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago',
           'opp 15 wk year ago', 'opp 16 wk year ago', 'opp 17 wk year ago',
           'opp 18 wk year ago', 'opp 19 wk year ago', 'opp 20 wk year ago',
           'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
           'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago',
           'opp 27 wk year ago', 'opp 28 wk year ago', 'opp 29 wk year ago',
           'opp 30 wk year ago', 'opp 31 wk year ago', 'opp 32 wk year ago',
           'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago'])
        

        new_df2 = pd.merge(qb_df2,new_df, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        
        test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
        for i in range(1,36):
            #print(i)
            wks_ago = i
            week_num_wks_ago = "week num " + str(i) + " wks ago"
            year_wks_ago = "year " + str(i) + " wks ago"
            #player_wks_ago = "player " + str(i) + " wks ago"
            team_wk_year = 'team_wk_year ' + str(i) + " wks ago"
            if(i == 1):
                curr_wk = "week_num"
                curr_yr = "year"
                test_df[curr_wk] = test_df['week_num']
                test_df[curr_yr] = test_df['year']
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
            else:
                curr_wk = "week num " + str(i-1) + " wks ago"
                curr_yr = "year " + str(i-1) + " wks ago"
                test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
                test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])

            test_df[team_wk_year] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

        #test_df.head()
        new_def_df = pd.DataFrame()

        for i in range(0,36):
            #print(i)
            if (i == 0):
                #new_df = df_qb2
                test_df = test_df.drop(columns = ['CITY','week_num','year'])
                new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
              #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
                print(new_def_df.head())
            else:
                temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
              #  temp_df2 = temp_df
                for col in temp_df.columns:
                    new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                    temp_df[new_col] = temp_df[col]
                    temp_df = temp_df.drop(columns = [col])
                #merge_column = "player " + str(i) + " wks ago"
                #merge_column = 
                merge_column = "team_wk_year " + str(i) + " wks ago"
                temp_df[merge_column] = def_df['team_wk_year']
                #temp_df['team_wk_year'] = def_df['team_wk_year']
               # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
                new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
                #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
        new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
               'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
               'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
               'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
               'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
               'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
               'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
               'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
               'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
               'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
               'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
               'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
               'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
               'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
               'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
               'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
               'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
               'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
               'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
               'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
               'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
               'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
               'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
               'year 35 wks ago', 'Team', #'CITY',
                                                 'Week', 'Sack', 'FR', 'INT', 'DefTD',
               'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD', 'FPts', 'week_num',
               'year', #'Opp',
                                                 'away flag','opp_wk_year'
                                                ])
        
        
        print(df['opp wk year'])
        print(df.shape)
        print(new_df2.shape)
        temp = df[['opp wk year','player_wk_year']]
        new_df2 = new_df2.drop(columns = ['opp wk year'])
        new_df2 = pd.merge(new_df2, temp, how = 'left', on = 'player_wk_year', suffixes = (None,None))
        new_df2 = new_df2.drop(columns = ['CITY', 'Opp'])
       # new_df2['opp wk year'] = df['opp wk year']
        new_df2 = new_df2.drop_duplicates()
        new_df3 = pd.merge(new_df2,new_def_df2,how = 'left', left_on = ['opp wk year'], right_on = ['team_wk_year'], suffixes = (None,None))
        
        new_df3 = new_df3.drop(columns = ['def matchup opp_wk_year 10wks ago', 'def matchup opp_wk_year 11wks ago',
             'def matchup opp_wk_year 12wks ago', 'def matchup opp_wk_year 13wks ago',
             'def matchup opp_wk_year 14wks ago', 'def matchup opp_wk_year 15wks ago',
             'def matchup opp_wk_year 16wks ago', 'def matchup opp_wk_year 17wks ago',
             'def matchup opp_wk_year 18wks ago', 'def matchup opp_wk_year 19wks ago',
             'def matchup opp_wk_year 1wks ago', 'def matchup opp_wk_year 20wks ago',
             'def matchup opp_wk_year 21wks ago', 'def matchup opp_wk_year 22wks ago',
             'def matchup opp_wk_year 23wks ago', 'def matchup opp_wk_year 24wks ago',
             'def matchup opp_wk_year 25wks ago', 'def matchup opp_wk_year 26wks ago',
             'def matchup opp_wk_year 27wks ago', 'def matchup opp_wk_year 28wks ago',
             'def matchup opp_wk_year 29wks ago', 'def matchup opp_wk_year 2wks ago',
             'def matchup opp_wk_year 30wks ago', 'def matchup opp_wk_year 31wks ago',
             'def matchup opp_wk_year 32wks ago', 'def matchup opp_wk_year 33wks ago',
             'def matchup opp_wk_year 34wks ago', 'def matchup opp_wk_year 35wks ago',
             'def matchup opp_wk_year 3wks ago', 'def matchup opp_wk_year 4wks ago',
             'def matchup opp_wk_year 5wks ago', 'def matchup opp_wk_year 6wks ago',
             'def matchup opp_wk_year 7wks ago', 'def matchup opp_wk_year 8wks ago',
             'def matchup opp_wk_year 9wks ago', 'opp opp_wk_year 10wks ago',
             'opp opp_wk_year 11wks ago', 'opp opp_wk_year 12wks ago',
             'opp opp_wk_year 13wks ago', 'opp opp_wk_year 14wks ago',
             'opp opp_wk_year 15wks ago', 'opp opp_wk_year 16wks ago',
             'opp opp_wk_year 17wks ago', 'opp opp_wk_year 18wks ago',
             'opp opp_wk_year 19wks ago', 'opp opp_wk_year 1wks ago',
             'opp opp_wk_year 20wks ago', 'opp opp_wk_year 21wks ago',
             'opp opp_wk_year 22wks ago', 'opp opp_wk_year 23wks ago',
             'opp opp_wk_year 24wks ago', 'opp opp_wk_year 25wks ago',
             'opp opp_wk_year 26wks ago', 'opp opp_wk_year 27wks ago',
             'opp opp_wk_year 28wks ago', 'opp opp_wk_year 29wks ago',
             'opp opp_wk_year 2wks ago', 'opp opp_wk_year 30wks ago',
             'opp opp_wk_year 31wks ago', 'opp opp_wk_year 32wks ago',
             'opp opp_wk_year 33wks ago', 'opp opp_wk_year 34wks ago',
             'opp opp_wk_year 35wks ago', 'opp opp_wk_year 3wks ago',
             'opp opp_wk_year 4wks ago', 'opp opp_wk_year 5wks ago',
             'opp opp_wk_year 6wks ago' ,'opp opp_wk_year 7wks ago',
             'opp opp_wk_year 8wks ago', 'opp opp_wk_year 9wks ago', 'opp_wk_year'])
        
        #new_df3['week_match'] = new_df3['CITY'] + " " + new_df3['Opp']
        
       # nfl_sched = nfl_sched[(nfl_sched['Year'] == year) & (nfl_sched['Week'] == week)]
        nfl_sched['Opponent'] = nfl_sched["Opponent"].str.replace("@","")
        nfl_sched['week_match'] = nfl_sched['TEAM'] + " " + nfl_sched['Opponent']
        nfl_sched = nfl_sched[['week_match']]
        
        #new_df4 = pd.merge(new_df3,nfl_sched,how = 'inner', on = ['week_match'], suffixes = (None,None))
        #new_df4 = new_df4.drop(columns = ['week_match'])
        
        
        new_df3.to_csv(save_file)
        
        return new_df3


In [29]:
def def_feat_eng_df_creation(def_df,
                                 file_save):
                
    def_df['Opp'] = def_df["Opp"].str.replace("@","")
    def_df['opp_wk_year'] = def_df['Opp'] + " " + def_df['week_num'].astype(str) + " " + def_df['year'].astype(str)    

    tms = ['Tm','Tm 1 wks ago', 'Tm 2 wks ago',
              'Tm 3 wks ago','Tm 4 wks ago','Tm 5 wks ago','Tm 6 wks ago',
              'Tm 7 wks ago','Tm 8 wks ago','Tm 9 wks ago','Tm 10 wks ago',
              'Tm 11 wks ago','Tm 12 wks ago','Tm 13 wks ago','Tm 14 wks ago',
              'Tm 15 wks ago','Tm 16 wks ago','Tm 17 wks ago','Tm 18 wks ago',
              'Tm 19 wks ago','Tm 20 wks ago','Tm 21 wks ago','Tm 22 wks ago',
              'Tm 23 wks ago','Tm 24 wks ago','Tm 25 wks ago','Tm 26 wks ago',
              'Tm 27 wks ago','Tm 28 wks ago','Tm 29 wks ago','Tm 30 wks ago',
              'Tm 31 wks ago','Tm 32 wks ago','Tm 33 wks ago','Tm 34 wks ago',
              'Tm 35 wks ago']

    opp = ['Opp','Opp 1 wks ago', 'Opp 2 wks ago',
              'Opp 3 wks ago','Opp 4 wks ago','Opp 5 wks ago','Opp 6 wks ago',
              'Opp 7 wks ago','Opp 8 wks ago','Opp 9 wks ago','Opp 10 wks ago',
              'Opp 11 wks ago','Opp 12 wks ago','Opp 13 wks ago','Opp 14 wks ago',
              'Opp 15 wks ago','Opp 16 wks ago','Opp 17 wks ago','Opp 18 wks ago',
              'Opp 19 wks ago','Opp 20 wks ago','Opp 21 wks ago','Opp 22 wks ago',
              'Opp 23 wks ago','Opp 24 wks ago','Opp 25 wks ago','Opp 26 wks ago',
              'Opp 27 wks ago','Opp 28 wks ago','Opp 29 wks ago','Opp 30 wks ago',
              'Opp 31 wks ago','Opp 32 wks ago','Opp 33 wks ago','Opp 34 wks ago',
              'Opp 35 wks ago']

        #opp
    opp_wk_year = ['opp wk year', 'opp 1 wk year ago', 'opp 2 wk year ago', 'opp 3 wk year ago',
              'opp 4 wk year ago', 'opp 5 wk year ago', 'opp 6 wk year ago', 'opp 7 wk year ago',
              'opp 8 wk year ago', 'opp 9 wk year ago', 'opp 10 wk year ago', 'opp 11 wk year ago',
              'opp 12 wk year ago', 'opp 13 wk year ago', 'opp 14 wk year ago', 'opp 15 wk year ago',
              'opp 16 wk year ago', 'opp 17 wk year ago', 'opp 18 wk year ago', 'opp 19 wk year ago',
              'opp 20 wk year ago', 'opp 21 wk year ago', 'opp 22 wk year ago', 'opp 23 wk year ago',
              'opp 24 wk year ago', 'opp 25 wk year ago', 'opp 26 wk year ago', 'opp 27 wk year ago',
              'opp 28 wk year ago', 'opp 29 wk year ago', 'opp 30 wk year ago', 'opp 31 wk year ago',
              'opp 32 wk year ago', 'opp 33 wk year ago', 'opp 34 wk year ago', 'opp 35 wk year ago']

    wk = ['Week', 'Week 1 wks ago', 'Week 2 wks ago', 'Week 3 wks ago', 'Week 4 wks ago', 'Week 5 wks ago',
              'Week 6 wks ago', 'Week 7 wks ago', 'Week 8 wks ago', 'Week 9 wks ago', 'Week 10 wks ago',
              'Week 11 wks ago', 'Week 12 wks ago', 'Week 13 wks ago', 'Week 14 wks ago', 'Week 15 wks ago',
              'Week 16 wks ago', 'Week 17 wks ago', 'Week 18 wks ago', 'Week 19 wks ago', 'Week 20 wks ago',
              'Week 21 wks ago', 'Week 22 wks ago', 'Week 23 wks ago', 'Week 24 wks ago', 'Week 25 wks ago',
              'Week 26 wks ago', 'Week 27 wks ago', 'Week 28 wks ago', 'Week 29 wks ago', 'Week 30 wks ago',
              'Week 31 wks ago', 'Week 32 wks ago', 'Week 33 wks ago', 'Week 34 wks ago', 'Week 35 wks ago',
              'Week 36 wks ago']

    year = ['Year', 'year 1 wks ago', 'year 2 wks ago', 'year 3 wks ago', 'year 4 wks ago', 'year 5 wks ago',
               'year 6 wks ago', 'year 7 wks ago', 'year 8 wks ago', 'year 9 wks ago', 'year 10 wks ago',
               'year 11 wks ago', 'year 12 wks ago', 'year 13 wks ago', 'year 14 wks ago', 'year 15 wks ago',
               'year 16 wks ago', 'year 17 wks ago', 'year 18 wks ago', 'year 19 wks ago', 'year 20 wks ago',
               'year 21 wks ago', 'year 22 wks ago', 'year 23 wks ago', 'year 24 wks ago', 'year 25 wks ago',
               'year 26 wks ago', 'year 27 wks ago', 'year 28 wks ago', 'year 29 wks ago', 'year 30 wks ago',
               'year 31 wks ago', 'year 32 wks ago', 'year 33 wks ago', 'year 34 wks ago', 'year 35 wks ago',
               'year 36 wks ago']
        
    test_df = def_df[['CITY','week_num', 'year', 'team_wk_year']]
    for i in range(1,36):
        print(i)
        wks_ago = i
        week_num_wks_ago = "week num " + str(i) + " wks ago"
        year_wks_ago = "year " + str(i) + " wks ago"
        player_wks_ago = "team_wk_year " + str(i) + " wks ago"
        if(i == 1):
            curr_wk = "week_num"
            curr_yr = "year"
            test_df[curr_wk] = test_df['week_num']
            test_df[curr_yr] = test_df['year']
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
        else:
            curr_wk = "week num " + str(i-1) + " wks ago"
            curr_yr = "year " + str(i-1) + " wks ago"
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
        test_df[player_wks_ago] = test_df['CITY'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

            
    new_def_df = pd.DataFrame()

    for i in range(0,36):
        print(i)
        if (i == 0):
            #new_df = df_qb2
            test_df = test_df.drop(columns = ['CITY','week_num','year'])
            new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['team_wk_year'], suffixes = (None, None))
          #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
            print(new_def_df.head())
        else:
            temp_df = def_df.drop(columns = ['Team','CITY','Week','week_num','year'])
      #  temp_df2 = temp_df
            for col in temp_df.columns:
                new_col = 'def matchup' + " " + col + " " + str(i) + 'wks ago'
                temp_df[new_col] = temp_df[col]
                temp_df = temp_df.drop(columns = [col])
        #merge_column = "player " + str(i) + " wks ago"
        #merge_column = 
            merge_column = "team_wk_year " + str(i) + " wks ago"
            temp_df[merge_column] = def_df['team_wk_year']
        #temp_df['team_wk_year'] = def_df['team_wk_year']
       # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
            new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
        #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
    new_def_df2 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
           'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
           'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
           'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
           'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
           'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
           'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
           'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
           'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
           'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
           'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
           'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
           'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
           'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
           'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
           'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
           'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
           'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
           'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
           'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
           'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
           'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
           'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
           'year 35 wks ago', 'Team', 'CITY', 
                                #  'week_num',
                                   'Sack', 'FR', 'INT', 'DefTD',
           'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD',
                                         #'FPts', 
                                         'PA',
        #                                 'week_num',
       #'year'
                               #   , 'Opp',
                                         'away flag'
                                 ]) 
            
    test_df = def_df[['Opp','week_num', 'year', 'opp_wk_year']]
    for i in range(1,36):
        print(i)
        wks_ago = i
        week_num_wks_ago = "week num " + str(i) + " wks ago"
        year_wks_ago = "year " + str(i) + " wks ago"
        #player_wks_ago = "player " + str(i) + " wks ago"
        team_wk_year = 'opp_wk_year ' + str(i) + " wks ago"
        if(i == 1):
            curr_wk = "week_num"
            curr_yr = "year"
            test_df[curr_wk] = test_df['week_num']
            test_df[curr_yr] = test_df['year']
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
        else:
            curr_wk = "week num " + str(i-1) + " wks ago"
            curr_yr = "year " + str(i-1) + " wks ago"
            test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
                                             np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] < 2022), 17, test_df[curr_wk] - 1))
            test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])

        test_df[team_wk_year] = test_df['Opp'] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)

    #print(week_num_wks_ago)
#    for j in range(len(df_qb2)):
 #       if(i == 1):
 #           test_df[curr_wk] = test_df['Week']
 #           test_df[curr_yr] = test_df['Year']
 #           test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk] - 1)
 #           test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
 #       else:
 #           test_df[week_num_wks_ago] = np.where(test_df[curr_wk] == 1, 17, test_df[curr_wk]-1)
 #           test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr] -1, test_df[curr_yr])
            
 #       test_df[player_wks_ago] = test_df['Player '] + " " + str(test_df[week_num_wks_ago]) + " " + str(test_df[year_wks_ago])
            
            
    test_df.head()
    test_df = test_df.drop_duplicates()    
        
    new_def_df = pd.DataFrame()
    print("test")
    for i in range(0,36):
        print(i)
        if (i == 0):
        #new_df = df_qb2
            test_df = test_df.drop(columns = ['Opp','week_num','year'])
            new_def_df = pd.merge(def_df, test_df, how = 'left', on = ['opp_wk_year'], suffixes = (None, None))
      #  new_def_df = new_def_df.drop(columns = ['team_wk_year'])
            print(new_def_df.head())
        else:
            temp_df = def_df.drop(columns = ['Team','Opp','Week','week_num','year'])
      #  temp_df2 = temp_df
            for col in temp_df.columns:
                new_col = 'opp matchup' + " " + col + " " + str(i) + 'wks ago'
                temp_df[new_col] = temp_df[col]
                temp_df = temp_df.drop(columns = [col])
        #merge_column = "player " + str(i) + " wks ago"
        #merge_column = 
            merge_column = "opp_wk_year " + str(i) + " wks ago"
            temp_df[merge_column] = def_df['opp_wk_year']
        #temp_df['team_wk_year'] = def_df['team_wk_year']
       # qb_no_null = qb_no_null.drop(columns = [opp_wk_year[i]])
            new_def_df = pd.merge(new_def_df, temp_df, how = 'left', on = [merge_column], suffixes = (None, None))
            new_def_df = new_def_df.drop_duplicates(subset=merge_column, keep = 'first')
            print(new_def_df.shape)
        #new_def_df = new_def_df.drop(columns = ['team_wk_year'])

        
    new_def_df3 = new_def_df.drop(columns = ['week num 1 wks ago', 'year 1 wks ago', 'week num 2 wks ago',
       'year 2 wks ago', 'week num 3 wks ago', 'year 3 wks ago',
       'week num 4 wks ago', 'year 4 wks ago', 'week num 5 wks ago',
       'year 5 wks ago', 'week num 6 wks ago', 'year 6 wks ago',
       'week num 7 wks ago', 'year 7 wks ago', 'week num 8 wks ago',
       'year 8 wks ago', 'week num 9 wks ago', 'year 9 wks ago',
       'week num 10 wks ago', 'year 10 wks ago', 'week num 11 wks ago',
       'year 11 wks ago', 'week num 12 wks ago', 'year 12 wks ago',
       'week num 13 wks ago', 'year 13 wks ago', 'week num 14 wks ago',
       'year 14 wks ago', 'week num 15 wks ago', 'year 15 wks ago',
       'week num 16 wks ago', 'year 16 wks ago', 'week num 17 wks ago',
       'year 17 wks ago', 'week num 18 wks ago', 'year 18 wks ago',
       'week num 19 wks ago', 'year 19 wks ago', 'week num 20 wks ago',
       'year 20 wks ago', 'week num 21 wks ago', 'year 21 wks ago',
       'week num 22 wks ago', 'year 22 wks ago', 'week num 23 wks ago',
       'year 23 wks ago', 'week num 24 wks ago', 'year 24 wks ago',
       'week num 25 wks ago', 'year 25 wks ago', 'week num 26 wks ago',
       'year 26 wks ago', 'week num 27 wks ago', 'year 27 wks ago',
       'week num 28 wks ago', 'year 28 wks ago', 'week num 29 wks ago',
       'year 29 wks ago', 'week num 30 wks ago', 'year 30 wks ago',
       'week num 31 wks ago', 'year 31 wks ago', 'week num 32 wks ago',
       'year 32 wks ago', 'week num 33 wks ago', 'year 33 wks ago',
       'week num 34 wks ago', 'year 34 wks ago', 'week num 35 wks ago',
       'year 35 wks ago', 'Team', 'CITY', 'Week', 'Sack', 'FR', 'INT', 'DefTD',
       'PA', 'PaYD', 'RuYd', 'Safety', 'KickTD',
                                         'FPts', 'team_wk_year',
                                         'PA',
                                         'week_num',
       'year', 'Opp',
                                         'away flag'])
       
        
    new_def_df4 = pd.merge(new_def_df2, new_def_df3, how = 'inner', on = 'opp_wk_year', suffixes = (None,None))

   
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = new_def_df4[['team_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['opp matchup Sack', 'opp matchup FR', 'opp matchup INT', 'opp matchup DefTD', 'opp matchup PA',
            'opp matchup PaYD', 'opp matchup RuYd',
       'opp matchup Safety', 'opp matchup KickTD', 'opp matchup FPts', 'opp matchup away flag',
           'opp matchup PA']

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            new_def_df4[fillna_col] = new_def_df4[fillna_col].fillna(0)
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = new_def_df4[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     

        
    new_def_df4 = pd.merge(new_def_df4, temp_df, how = 'inner', on= 'team_wk_year', suffixes = (None,None))        
    
    
    trend = [2,3,4,5,6,9,12,15,18,24,30]
    temp_df = new_def_df4[['team_wk_year']]    
    temp_df2 = pd.DataFrame()     
    features = ['def matchup Sack',
    'def matchup FR',
    'def matchup INT',
    'def matchup DefTD',
    'def matchup PA',
    'def matchup PaYD',
    'def matchup RuYd',
    'def matchup Safety',
    'def matchup KickTD',
    'def matchup FPts',
    'def matchup Opp',
    'def matchup away flag',
           'def matchup PA']

    for feat in features:
        for i in range(1,35):
            fillna_col = feat + " " + str(i) + "wks ago"
            new_def_df4[fillna_col] = new_def_df4[fillna_col].fillna(0)
            newcol_i = "new col " + str(i)
            temp_df2[newcol_i] = new_def_df4[fillna_col]
        for t in trend:
        
            newcol_sum = "sum " + feat + " " + str(t) + "wk trend"
            newcol_avg = "avg " + feat + " " + str(t) + "wk trend"
            newcol_min = "min " + feat + " " + str(t) + "wk trend"
            newcol_max = "max " + feat + " " + str(t) + "wk trend"
            if (t == 2):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2']].max(skipna= False, axis = 1) 
            if (t == 3):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3']].max(skipna= False, axis = 1)
            if (t == 4):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2', 'new col 3','new col 4']].max(skipna= False, axis = 1)     
            if (t == 5):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3','new col 4', 'new col 5']].max(skipna= False, axis = 1)     
            if (t == 6):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6']].max(skipna= False, axis = 1)     
            if (t == 9):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9']].max(skipna= False, axis = 1)     
            if (t == 12):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12']].max(skipna= False, axis = 1)     
            if (t == 15):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15']].max(skipna= False, axis = 1)     
            if (t == 18):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18']].max(skipna= False, axis = 1)     
            if (t == 24):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24']].max(skipna= False, axis = 1)     
            if (t == 30):
                temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].sum(skipna= False, axis = 1) 
                temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].mean(skipna= False, axis = 1) 
                temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].min(skipna= False, axis = 1) 
                temp_df[newcol_max] = temp_df2[['new col 1', 'new col 2','new col 3',
                                         'new col 4', 'new col 5','new col 6',
                                        'new col 7', 'new col 8', 'new col 9',
                                        'new col 10', 'new col 11', 'new col 12',
                                        'new col 13', 'new col 14', 'new col 15',
                                        'new col 16','new col 17','new col 18',
                                        'new col 19', 'new col 20', 'new col 21',
                                        'new col 22', 'new col 23', 'new col 24',
                                        'new col 25','new col 26','new col 27',
                                        'new col 28', 'new col 29','new col 30']].max(skipna= False, axis = 1)     


    new_def_df4 = pd.merge(new_def_df4, temp_df, how = 'inner', on= 'team_wk_year', suffixes = (None,None))
    new_def_df4 = new_def_df4.drop_duplicates()
    new_def_df4.to_csv(file_save)
    
    return new_def_df4

In [30]:
def def_curr_wk_oot(def_df,
                    nfl_sched,
                    curr_wk,
                    curr_yr,
                    file_save):
    nfl_sched = nfl_sched[(nfl_sched['Year'] == curr_yr) & 
                         (nfl_sched['Week'] == curr_wk)]
    
    nfl_sched = nfl_sched.rename(columns = {'Year':'year',
                                           'TEAM':'CITY',
                                           'Opponent':'Opp'})
    nfl_sched['week_num'] = nfl_sched['Week']
    nfl_sched['Opp'] = nfl_sched["Opp"].str.replace("@","")
    nfl_sched['opp wk year'] = nfl_sched['Opp'] + " " + nfl_sched['week_num'].astype(str) + " " + nfl_sched['year'].astype(str)    
   

    nfl_sched['Team'] = nfl_sched['CITY'].map({'ARI':'Cardinals','SF':'49ers','NO':'Saints',
                                   'CAR':'Panthers','DAL':'Cowboys','HOU':'Texans',
                                   'LV':'Raiders','LAR':'Rams','DET':'Lions',
                                   'KC':'Chiefs','WSH':'Commanders','BAL':'Ravens',
                                   'IND':'Colts','PIT':'Steelers','CIN':'Bengals',
                                   'SEA':'Seahawks','MIA':'Dolphins','MIN':'Vikings',
                                   'NE':'Patriots','NYG':'Giants','TEN':'Titans',
                                   'DEN':'Broncos', 'NYJ':'Jets','PHI':'Eagles',
                                   'LAC':'Chargers','TB':'Buccaneers','BUF':'Bills',
                                   'CLE':'Browns','ATL':'Falcons','JAX':'Jaguars',
                                   'CHI':'Bears','GB':'Packers'})
    
    nfl_sched['Sack'] = None
    nfl_sched['FR'] = None
    nfl_sched['INT'] = None
    nfl_sched['DefTD'] = None
    nfl_sched['PA'] = None
    nfl_sched['PaYD'] = None
    nfl_sched['RuYd'] = None
    nfl_sched['Safety'] = None
    nfl_sched['KickTD'] = None
    nfl_sched['FPts'] = None
    nfl_sched['away flag'] = None

    def_df['Opp'] = def_df['Opp'].str.replace('@','')
    def_df['opp wk year'] = def_df['Opp'] + " " + def_df['week_num'].astype(str) + " " + def_df['year'].astype(str)
    #def_df = def_df.append(nfl_sched)
    
    def_df = def_df.append(nfl_sched)
    def_df = def_df.drop(columns = ['opp wk year'])
    
    def_df_wk = def_feat_eng_df_creation(def_df = def_df,
                                 file_save = file_save)
    
    filt_wk = str(curr_wk) + " " + str(curr_yr)
    
    def_df_wk = def_df_wk[def_df_wk['team_wk_year'].str.contains(filt_wk)]
    
    #def_df_wk.to_cvs(file_save)
    
    return def_df_wk

In [31]:
def def_curr_wk_for_positions(def_df,
                    nfl_sched,
                    curr_wk,
                    curr_yr):
    nfl_sched = nfl_sched[(nfl_sched['Year'] == curr_yr) & 
                         (nfl_sched['Week'] == curr_wk)]
    
    nfl_sched = nfl_sched.rename(columns = {'Year':'year',
                                           'TEAM':'CITY',
                                           'Opponent':'Opp'})
    nfl_sched['week_num'] = nfl_sched['Week']
    nfl_sched['Opp'] = nfl_sched["Opp"].str.replace("@","")
    nfl_sched['opp_wk_year'] = nfl_sched['Opp'] + " " + nfl_sched['week_num'].astype(str) + " " + nfl_sched['year'].astype(str)    
   

    nfl_sched['Team'] = nfl_sched['CITY'].map({'ARI':'Cardinals','SF':'49ers','NO':'Saints',
                                   'CAR':'Panthers','DAL':'Cowboys','HOU':'Texans',
                                   'LV':'Raiders','LAR':'Rams','DET':'Lions',
                                   'KC':'Chiefs','WSH':'Commanders','BAL':'Ravens',
                                   'IND':'Colts','PIT':'Steelers','CIN':'Bengals',
                                   'SEA':'Seahawks','MIA':'Dolphins','MIN':'Vikings',
                                   'NE':'Patriots','NYG':'Giants','TEN':'Titans',
                                   'DEN':'Broncos', 'NYJ':'Jets','PHI':'Eagles',
                                   'LAC':'Chargers','TB':'Buccaneers','BUF':'Bills',
                                   'CLE':'Browns','ATL':'Falcons','JAX':'Jaguars',
                                   'CHI':'Bears','GB':'Packers'})
    
    nfl_sched['Sack'] = None
    nfl_sched['FR'] = None
    nfl_sched['INT'] = None
    nfl_sched['DefTD'] = None
    nfl_sched['PA'] = None
    nfl_sched['PaYD'] = None
    nfl_sched['RuYd'] = None
    nfl_sched['Safety'] = None
    nfl_sched['KickTD'] = None
    nfl_sched['FPts'] = None
    nfl_sched['away flag'] = None

    def_df['Opp'] = def_df['Opp'].str.replace('@','')
    def_df['opp_wk_year'] = def_df['Opp'] + " " + def_df['week_num'].astype(str) + " " + def_df['year'].astype(str)
    def_df = def_df.append(nfl_sched)

#    filt_wk = str(curr_wk) + " " + str(curr_yr)
    
#    def_df_wk = def_df_wk[def_df_wk['team_wk_year'].str.contains(filt_wk)]
    
    return def_df

In [12]:
test_fun = update_player_weekly_stats()

C:\Users\RobertBruffey\anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])
<ipython-input-3-a631318dc75d>:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['Playing_Prd_End'] = players['Playing_Prd_End'].astype(int)
<ipython-input-3-a631318dc75d>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [13]:
nfl_sched = pd.read_csv('C:/Users/RobertBruffey/OneDrive - Blend 360/Documents/The Plan/NFL/NFL Schedule.csv')
test = def_weekly_stats_scrape(first_yr = 2015, last_yr = 2022, nfl_sched = nfl_sched)

In [14]:
nfl_sched[(nfl_sched['Year'] == 2022) & (nfl_sched['Week'] == 15)]

,Year,Week,TEAM,Opponent,team_wk_year
448,2022,15,ARI,@DEN,ARI 15 2022
449,2022,15,ATL,@NO,ATL 15 2022
450,2022,15,BAL,@CLE,BAL 15 2022
451,2022,15,BUF,MIA,BUF 15 2022
452,2022,15,CAR,PIT,CAR 15 2022
453,2022,15,CHI,PHI,CHI 15 2022
454,2022,15,CIN,@TB,CIN 15 2022
455,2022,15,CLE,BAL,CLE 15 2022
456,2022,15,DAL,@JAX,DAL 15 2022
457,2022,15,DEN,ARI,DEN 15 2022


In [15]:
def_weekly = def_curr_wk_for_positions(def_df = test,
                    nfl_sched = nfl_sched,
                    curr_wk = 15,
                    curr_yr = 2022)

In [16]:
def_weekly.head()

,Team,CITY,Week,Sack,FR,INT,DefTD,PA,PaYD,RuYd,Safety,KickTD,FPts,week_num,year,Opp,away flag,team_wk_year,opp_wk_year
0,Seahawks,SEA,1,2,3,0,1,34,297.0,76.0,0,1,20.0,1,2015,LAR,1,SEA 1 2015,LAR 1 2015
1,Panthers,CAR,1,5,1,2,1,9,183.0,96.0,0,0,17.0,1,2015,JAX,1,CAR 1 2015,JAX 1 2015
2,Rams,LAR,1,6,0,1,0,31,251.0,124.0,0,1,14.0,1,2015,SEA,0,LAR 1 2015,SEA 1 2015
3,Titans,TEN,1,4,0,2,1,14,210.0,92.0,0,0,14.0,1,2015,TB,1,TEN 1 2015,TB 1 2015
4,Lions,DET,1,2,1,2,1,33,404.0,95.0,0,0,14.0,1,2015,LAC,1,DET 1 2015,LAC 1 2015


In [17]:
test.head()

,Team,CITY,Week,Sack,FR,INT,DefTD,PA,PaYD,RuYd,Safety,KickTD,FPts,week_num,year,Opp,away flag,team_wk_year,opp_wk_year
0,Seahawks,SEA,1,2,3,0,1,34,297.0,76.0,0,1,20.0,1,2015,LAR,1,SEA 1 2015,LAR 1 2015
1,Panthers,CAR,1,5,1,2,1,9,183.0,96.0,0,0,17.0,1,2015,JAX,1,CAR 1 2015,JAX 1 2015
2,Rams,LAR,1,6,0,1,0,31,251.0,124.0,0,1,14.0,1,2015,SEA,0,LAR 1 2015,SEA 1 2015
3,Titans,TEN,1,4,0,2,1,14,210.0,92.0,0,0,14.0,1,2015,TB,1,TEN 1 2015,TB 1 2015
4,Lions,DET,1,2,1,2,1,33,404.0,95.0,0,0,14.0,1,2015,LAC,1,DET 1 2015,LAC 1 2015


In [20]:
df_qb = position_feat_eng_df_creation(player_df = test_fun,
                              def_df = test,
                              position = 'QB',
                                     file_save = 'qb_player_data_v1.csv')

0
Brandon Allen
1
Josh Allen
2
Kyle Allen
3
Derek Anderson
4
Matt Barkley
5
Charlie Batch
6
C.J. Beathard
7
Kurt Benkert
8
David Blough
9
Ian Book
10
Blake Bortles
11
Trevone Boykin
12
Tim Boyle
13
Sam Bradford
14
Tom Brady
15
Tyler Bray
16
Drew Brees
17
Teddy Bridgewater
18
Jacoby Brissett
19
Anthony Brown
20
Joe Burrow
21
Joe Callahan
22
Jason Campbell
23
David Carr
24
Derek Carr
25
Matt Cassel
26
Jimmy Clausen
27
Kellen Clemens
28
Connor Cook
29
Kirk Cousins
30
Jay Cutler
31
Andy Dalton
32
Chase Daniel
33
B.J. Daniels
34
Sam Darnold
35
Austin Davis
36
Dominique Davis
37
Ben DiNucci
38
Joshua Dobbs
39
Jeff Driskel
40
Jacob Eason
41
Armanti Edwards
42
Trent Edwards
43
Sam Ehlinger
44
David Fales
45
Luke Falk
46
Justin Fields
47
Ryan Finley
48
Ryan Fitzpatrick
49
Joe Flacco
50
Matt Flynn
51
Nick Foles 
52
Feleipe Franks
53
Josh Freeman
54
Jake Fromm
55
Blaine Gabbert
56
Jimmy Garoppolo
57
Garrett Gilbert
58
Mike Glennon
59
Jared Goff
60
Bruce Gradkowski
61
Will Grier
62
Robert Griffin 

<ipython-input-19-96230670df94>:441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Week'] = test_df['Week'].astype(int)
<ipython-input-19-96230670df94>:442: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Year'] = test_df['Year'].astype(int)
<ipython-input-19-96230670df94>:452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


<ipython-input-19-96230670df94>:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
<ipython-input-19-96230670df94>:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
<ipython-input-19-96230670df94>:466: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
          player_wk_year  week num 1 wks ago  year 1 wks ago  \
0   Brandon Allen 9 2019                   8            2019   
1  Brandon Allen 11 2019                  10            2019   
2  Brandon Allen 12 2019                  11            2019   
3  Brandon Allen 12 2020                  11            2020   
4  Brandon Allen 13 2020                  12            2020   

        player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0   Brandon Allen 8 2019                   7            2019   
1  Brandon Allen 10 2019                   9            2019   
2  Brandon Allen 11 2019                  10            2019   
3  Brandon Allen 11 2020                  10            2020   
4  Brandon Allen 12 2020                  11            2020   

        player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0   Brandon Allen 7 2019                   6            2019   
1   Brandon Allen 9 2019                   8            20

<ipython-input-19-96230670df94>:658: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-19-96230670df94>:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-19-96230670df94>:660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [21]:
df_qb.shape

(8348, 2869)

In [22]:
test_final = create_qb_trend_var(qb_df=df_qb,
                                 position = 'QB',
                                file_save = 'qb_player_data_final.csv')

(8348, 2694)
trend 1 creation


<ipython-input-7-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-7-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-7-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

merge 1
(8348, 3178)
trend 2 creation


<ipython-input-7-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-7-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-7-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 2
(8348, 4410)
trend 3 creation


<ipython-input-7-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-7-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-7-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 3
(8348, 4894)


In [23]:
def_weekly

,Team,CITY,Week,Sack,FR,INT,DefTD,PA,PaYD,RuYd,Safety,KickTD,FPts,week_num,year,Opp,away flag,team_wk_year,opp_wk_year
0,Seahawks,SEA,1,2,3,0,1,34,297.0,76.0,0,1,20.0,1,2015,LAR,1,SEA 1 2015,LAR 1 2015
1,Panthers,CAR,1,5,1,2,1,9,183.0,96.0,0,0,17.0,1,2015,JAX,1,CAR 1 2015,JAX 1 2015
2,Rams,LAR,1,6,0,1,0,31,251.0,124.0,0,1,14.0,1,2015,SEA,0,LAR 1 2015,SEA 1 2015
3,Titans,TEN,1,4,0,2,1,14,210.0,92.0,0,0,14.0,1,2015,TB,1,TEN 1 2015,TB 1 2015
4,Lions,DET,1,2,1,2,1,33,404.0,95.0,0,0,14.0,1,2015,LAC,1,DET 1 2015,LAC 1 2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,49ers,SF,15,None,None,None,None,None,None,None,None,None,None,15,2022,SEA,None,SF 15 2022,SEA 15 2022
476,Seahawks,SEA,15,None,None,None,None,None,None,None,None,None,None,15,2022,SF,None,SEA 15 2022,SF 15 2022
477,Buccaneers,TB,15,None,None,None,None,None,None,None,None,None,None,15,2022,CIN,None,TB 15 2022,CIN 15 2022
478,Titans,TEN,15,None,None,None,None,None,None,None,None,None,None,15,2022,LAC,None,TEN 15 2022,LAC 15 2022


In [24]:
test_fun

,Gamelog,Player,Position,Playing_Prd_Strt,Playing_Prd_End
0,https://www.pro-football-reference.com/players...,Isaako Aaitui,DT,2013,2013
4,https://www.pro-football-reference.com/players...,Jared Abbrederis,WR,2015,2017
6,https://www.pro-football-reference.com/players...,Mehdi Abdesmad,DE,2016,2016
8,https://www.pro-football-reference.com/players...,Isa Abdul-Quddus,S,2011,2016
9,https://www.pro-football-reference.com/players...,Ameer Abdullah,RB,2015,2022
...,...,...,...,...,...
27095,https://www.pro-football-reference.com/players...,Greg Zuerlein,K,2012,2022
27098,https://www.pro-football-reference.com/players...,Jabari Zuniga,DL,2020,2022
27101,https://www.pro-football-reference.com/players...,Jeremy Zuttah,G,2008,2016
27104,https://www.pro-football-reference.com/players...,Brandon Zylstra,WR,2018,2022


In [32]:
curr_wk_df = create_curr_wk_oot_qb(nfl_sched = nfl_sched,
                                   year= 2022, 
                                   week = 15,
                                   player_df=test_fun,
                                   position = 'QB',
                                  def_df = def_weekly,
                                  save_file = '2022qbweek15.csv')

0
Josh Allen
1
Kyle Allen
2
C.J. Beathard
3
Tom Brady
4
Teddy Bridgewater
5
Jacoby Brissett
6
Anthony Brown
7
Joe Burrow
8
Derek Carr
9
Kirk Cousins
10
Andy Dalton
11
Chase Daniel
12
Sam Darnold
13
Jeff Driskel
14
Jacob Eason
15
Sam Ehlinger
16
Justin Fields
17
Joe Flacco
18
Nick Foles 
19
Feleipe Franks
20
Jimmy Garoppolo
21
Jared Goff
22
Taylor Heinicke
23
Chad Henne
24
Justin Herbert
25
Taysom Hill
26
Brian Hoyer
27
Tyler Huntley
28
Jalen Hurts
29
Lamar Jackson
30
Josh Johnson
31
Daniel Jones
32
Mac Jones
33
Case Keenum
34
Trey Lance
35
Trevor Lawrence
36
Jordan Love
37
Patrick Mahomes
38
Marcus Mariota
39
Baker Mayfield
40
Colt McCoy
41
Trace McSorley
42
Davis Mills
43
Gardner Minshew II
44
Nick Mullens
45
Kyler Murray
46
Bryce Perkins
47
Kenny Pickett
48
Dak Prescott
49
Brock Purdy
50
Aaron Rodgers
51
Cooper Rush
52
Matt Ryan
53
Brett Rypien
54
Trevor Siemian
55
Geno Smith
56
Matthew Stafford
57
Jarrett Stidham
58
Nate Sudfeld
59
Tua Tagovailoa
60
Ryan Tannehill
61
Tyrod Taylor
62

<ipython-input-28-1f29e38a0d97>:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['Week']
<ipython-input-28-1f29e38a0d97>:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['Year']
<ipython-input-28-1f29e38a0d97>:513: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

0
              player_wk_year  week num 1 wks ago  year 1 wks ago  \
0         Josh Allen 15 2022                  14            2022   
1       Josh Johnson 15 2022                  14            2022   
2        Case Keenum 15 2022                  14            2022   
3       Tyrod Taylor 15 2022                  14            2022   
4  Mitchell Trubisky 15 2022                  14            2022   

            player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0         Josh Allen 14 2022                  13            2022   
1       Josh Johnson 14 2022                  13            2022   
2        Case Keenum 14 2022                  13            2022   
3       Tyrod Taylor 14 2022                  13            2022   
4  Mitchell Trubisky 14 2022                  13            2022   

            player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0         Josh Allen 13 2022                  12            2022   
1       Josh Johnson 13 2022                

<ipython-input-28-1f29e38a0d97>:715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-28-1f29e38a0d97>:716: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-28-1f29e38a0d97>:717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [33]:
print(curr_wk_df.shape)
a = np.setdiff1d(curr_wk_df.columns, df_qb.columns)
print (a)


(178, 2798)
[]


In [ ]:
#curr_wk_df['temp_col'] = curr_wk_df['CITY'] + " 5 2022"
#curr_wk_df = curr_wk_df[curr_wk_df['temp_col'] == curr_wk_df['opp wk year']]
#curr_wk_df = curr_wk_df.drop(columns = ['temp_col'])
#curr_wk_df = curr_wk_df.drop_duplicates()

In [34]:
qb_df_wk15 = create_qb_trend_var(qb_df = curr_wk_df,
                                position = 'QB',
                               file_save = '2022week15qb_final.csv')

(178, 2623)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(178, 3107)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 2
(178, 4339)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(750, 4823)


In [ ]:
#temp = qb_df_wk4.drop_duplicates(subset = 'player_wk_year', keep='first')

In [ ]:

#qb_df_wk4.to_csv('2022week4qb_final.csv')

In [35]:
rb_test = position_feat_eng_df_creation(player_df = test_fun,
                              def_df = test,
                              position = 'RB',
                                   file_save = 'rb_player_data_v1.csv')

0
Ameer Abdullah
1
Josh Adams
2
Bralon Addison
3
Salvon Ahmed
4
Jay Ajayi
5
Cam Akers
6
Anthony Allen
7
Armando Allen
8
Javorius Allen
9
Tyler Allgeier
10
C.J. Anderson
11
Antonio Andrews
12
Ryquell Armstead
13
Cameron Artis-Payne
14
George Atkinson
15
Edwin Baker
16
Lance Ball
17
Montee Ball
18
Kalen Ballage
19
Vick Ballard
20
Joe Banyard
21
Peyton Barber
22
Jake Bargas
23
Saquon Barkley
24
Kenjon Barner
25
Baron Batch
26
Jackie Battle
27
Nick Bawden
28
Joique Bell
29
Kahlil Bell
30
Le'Veon Bell
31
LeVante Bellamy
32
Eno Benjamin
33
Cedric Benson
34
Giovani Bernard
35
Kapri Bibbs
36
Raheem Blackshear
37
Khari Blasingame
38
LeGarrette Blount
39
Alfred Blue
40
Brandon Bolden
41
Reggie Bonnafon
42
Devontae Booker
43
Mike Boone
44
Allen Bradford
45
Ahmad Bradshaw
46
Darius Bradwell
47
Matt Breida
48
Gary Brightwell
49
Curtis Brinkley
50
Tony Brooks-James
51
Andre Brown
52
Brittain Brown
53
Bryce Brown
54
Donald Brown
55
Mack Brown
56
Malcolm Brown
57
Ronnie Brown
58
Spencer Brown
59
Rex B

462
Robert Turbin
463
De'Lance Turner
464
Michael Turner
465
Tyler Varga
466
Josh Vaughan
467
Ke'Shawn Vaughn
468
Shane Vereen
469
Kenneth Walker III
470
Austin Walter
471
Mark Walton
472
Jonathan Ward
473
Terron Ward
474
Danny Ware
475
Spencer Ware
476
Jaylen Warren
477
Michael Warren
478
DeAndre Washington
479
Dwayne Washington
480
Leon Washington
481
Terrell Watson
482
Trey Watts
483
Brandon Wegher
484
Beanie Wells
485
Charcandrick West
486
Terrance West
487
James White
488
Johnny White
489
Rachaad White
490
Zamir White
491
Fozzy Whittaker
492
Kerrith Whyte Jr
493
Brandon Wilds
494
Jordan Wilkins
495
Andre Williams
496
Antonio Williams
497
Damien Williams
498
Darrel Williams
499
David Williams
500
DeAngelo Williams
501
Dexter Williams
502
Jamaal Williams
503
Jarveon Williams
504
Javonte Williams
505
Jonathan Williams
506
Karlos Williams
507
Keiland Williams
508
Kerwynn Williams
509
Kyren Williams
510
Pooka Williams
511
Ryan Williams
512
Trayveon Williams
513
Trey Williams
514
Ty'Son

<ipython-input-26-96230670df94>:441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Week'] = test_df['Week'].astype(int)
<ipython-input-26-96230670df94>:442: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Year'] = test_df['Year'].astype(int)
<ipython-input-26-96230670df94>:452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
           player_wk_year  week num 1 wks ago  year 1 wks ago  \
0   Ameer Abdullah 2 2015                   1            2015   
1   Ameer Abdullah 4 2015                   3            2015   
2   Ameer Abdullah 6 2015                   5            2015   
3  Ameer Abdullah 12 2015                  11            2015   
4  Ameer Abdullah 13 2015                  12            2015   

         player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0   Ameer Abdullah 1 2015                  17            2014   
1   Ameer Abdullah 3 2015                   2            2015   
2   Ameer Abdullah 5 2015                   4            2015   
3  Ameer Abdullah 11 2015                  10            2015   
4  Ameer Abdullah 12 2015                  11            2015   

         player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0  Ameer Abdullah 17 2014                  16            20

<ipython-input-26-96230670df94>:658: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-26-96230670df94>:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-26-96230670df94>:660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [36]:
rb_test_final = create_qb_trend_var(qb_df=rb_test,
                                 position = 'RB',
                                file_save = 'rb_player_data_final.csv')

(8115, 2514)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(8115, 2998)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

<ipython-input-27-2de3c4070737>:352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2','new col 3',
<ipython-input-27-2de3c4070737>:356: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
<ipython-input-27-2de3c4070737>:360: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

merge 2
(8115, 4450)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(8115, 4934)


In [37]:
curr_wk_rd_df = create_curr_wk_oot_qb(nfl_sched = nfl_sched,
                                   year= 2022, 
                                   week = 15,
                                   player_df=test_fun,
                                   position = 'RB',
                                  def_df = def_weekly,
                                  save_file = '2022rbweek15.csv')

0
Ameer Abdullah
1
Salvon Ahmed
2
Cam Akers
3
Tyler Allgeier
4
Saquon Barkley
5
Eno Benjamin
6
Giovani Bernard
7
Raheem Blackshear
8
Khari Blasingame
9
Brandon Bolden
10
Mike Boone
11
Matt Breida
12
Gary Brightwell
13
Brittain Brown
14
Malcolm Brown
15
Spencer Brown
16
Rex Burkhead
17
Trenton Cannon
18
Michael Carter
19
Ty Chandler
20
Julius Chestnut
21
Nick Chubb
22
Corey Clement
23
Tevin Coleman
24
James Conner
25
Snoop Conner
26
Dalvin Cook
27
James Cook
28
DeeJay Dallas
29
Malik Davis
30
Mike Davis
31
Tyrion Davis-Price
32
AJ Dillon 
33
J.K. Dobbins
34
Rico Dowdle
35
Kenyan Drake
36
Trestan Ebner
37
Chase Edmonds
38
Gus Edwards
39
Clyde Edwards-Helaire
40
Austin Ekeler
41
Ezekiel Elliott
42
Travis Etienne
43
Chris Evans
44
Darrynton Evans
45
Demetric Felton
46
Jerome Ford
47
D'Onta Foreman
48
Leonard Fournette
49
Jake Funk
50
Kenneth Gainwell
51
Myles Gaskin
52
Antonio Gibson 
53
Cullen Gillaspia
54
Melvin Gordon
55
Breece Hall
56
C.J. Ham
57
Damien Harris
58
Kevin Harris
59
Najee 

315
Christine Michael
316
Sony Michel
317
Lamar Miller
318
Elijah Mitchell
319
Joe Mixon
320
Taquan Mizzell
321
David Montgomery
322
Collin Mooney
323
Mewelde Moore
324
Knowshon Moreno
325
Alfred Morris
326
Zack Moss
327
Raheem Mostert
328
Marcus Murphy
329
Richard Murphy
330
DeMarco Murray
331
Latavius Murray
332
Gabe Nabers
333
Ryan Nall
334
Detrez Newsome
335
Kene Nwangwu
336
Chris Ogbonnaya
337
Dare Ogunbowale
338
Branden Oliver
339
Qadree Ollison
340
Sewo Olonilua
341
Montell Owens
342
Devine Ozigbo
343
Isiah Pacheco
344
Jalen Parmele
345
Jacques Patrick
346
Jaret Patterson
347
Isaiah Pead
348
Cedric Peerman
349
Elijhaa Penny
350
Rashaad Penny
351
La'Mical Perine
352
Samaje Perine
353
Paul Perkins
354
Senorise Perry
355
Adrian Peterson
356
Scottie Phillips
357
Artavis Pierce
358
Bernard Pierce
359
Dameon Pierce
360
Lousaka Polite
361
Chris Polk
362
Tony Pollard
363
Tauren Poole
364
Troymaine Pope
365
Bilal Powell
366
William Powell
367
Adam Prentice
368
Chris Pressley
369
D'Vonte 

yay
0
             player_wk_year  week num 1 wks ago  year 1 wks ago  \
0    Ameer Abdullah 15 2022                  14            2022   
1  Godwin Igwebuike 15 2022                  14            2022   
2    Justin Jackson 15 2022                  14            2022   
3        Ty Johnson 15 2022                  14            2022   
4    Craig Reynolds 15 2022                  14            2022   

           player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0    Ameer Abdullah 14 2022                  13            2022   
1  Godwin Igwebuike 14 2022                  13            2022   
2    Justin Jackson 14 2022                  13            2022   
3        Ty Johnson 14 2022                  13            2022   
4    Craig Reynolds 14 2022                  13            2022   

           player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0    Ameer Abdullah 13 2022                  12            2022   
1  Godwin Igwebuike 13 2022                  12       

<ipython-input-28-1f29e38a0d97>:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['Week']
<ipython-input-28-1f29e38a0d97>:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['Year']
<ipython-input-28-1f29e38a0d97>:513: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


<ipython-input-28-1f29e38a0d97>:715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-28-1f29e38a0d97>:716: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-28-1f29e38a0d97>:717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [38]:
#curr_wk_rd_df['temp_col'] = curr_wk_rd_df['CITY'] + " 5 2022"
#curr_wk_rd_df = curr_wk_rd_df[curr_wk_rd_df['temp_col'] == curr_wk_rd_df['opp wk year']]
#curr_wk_rd_df = curr_wk_rd_df.drop(columns = ['temp_col'])
#curr_wk_rd_df = curr_wk_rd_df.drop_duplicates()

In [39]:
rb_df_wk15 = create_qb_trend_var(qb_df = curr_wk_rd_df,
                                position = 'RB',
                               file_save = '2022week15rb_final.csv')

(271, 2443)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(271, 2927)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 2
(271, 4379)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(689, 4863)


In [ ]:
#rb_temp = rb_df_wk3.drop_duplicates(subset = 'player_wk_year', keep='first')

In [ ]:
#rb_temp.to_csv('2022week4rb_final.csv')

In [ ]:
#del rb_test_final, rb_test, temp, qb_df_wk3, curr_wk_df,test_final,df_qb, test2

In [40]:
wr_test = position_feat_eng_df_creation(player_df = test_fun,
                              def_df = test,
                              position = 'WR',
                                     file_save = 'wr_player_data_v1.csv')

0
Jared Abbrederis
1
Davante Adams
2
Joe Adams
3
Kris Adams
4
Rodney Adams
5
Quincy Adeboyejo
6
Nelson Agholor
7
Kamar Aiken
8
Brandon Aiyuk
9
Seyi Ajirotutu
10
Landen Akers
11
Danario Alexander
12
Maurice Alexander
13
Mario Alford
14
Keenan Allen
15
Geronimo Allison
16
Danny Amendola
17
Joe Anderson
18
Robbie Anderson
19
JJ Arcega-Whiteside
20
Dri Archer
21
Anthony Armstrong
22
Dan Arnold
23
Devin Aromashodu
24
Marcell Ateman
25
Tutu Atwell
26
David Ausberry
27
Miles Austin
28
Tavon Austin
29
Jason Avant
30
Donnie Avery
31
Demarcus Ayers
32
Andre Baccellia
33
Alex Bachman
34
Jeff Badet
35
Stedman Bailey
36
Kawaan Baker
37
Doug Baldwin
38
Jonathan Baldwin
39
Michael Bandy
40
Brandon Banks
41
Ramses Barden
42
Rashod Bateman
43
Phil Bates
44
Cameron Batson
45
Cole Beasley
46
Odell Beckham Jr.
47
Chad Beebe
48
Reggie Begelton
49
David Bell
50
Josh Bellamy
51
Kelvin Benjamin
52
Travis Benjamin
53
Arrelious Benn
54
Earl Bennett
55
Trinity Benson
56
Jared Bernhardt
57
Braxton Berrios
58
Stan

464
Racey McMath
465
Marvin McNutt
466
Paul McRoberts
467
Robert Meachem
468
Cameron Meredith
469
Kirk Merritt
470
D.K. Metcalf
471
Jakobi Meyers
472
Marken Michel
473
Jaydon Mickens
474
Anthony Miller
475
Braxton Miller
476
Scott Miller
477
Dax Milne
478
Denzel Mims
479
Malcolm Mitchell
480
Myron Mitchell
481
Steven Mitchell
482
Ifeanyi Momah
483
Donte Moncrief
484
D.J. Montgomery
485
Darnell Mooney
486
Chris Moore
487
D.J. Moore
488
David Moore
489
Denarius Moore
490
Elijah Moore
491
Evan Moore
492
J'Mon Moore
493
Jason Moore
494
Lance Moore
495
Marlon Moore
496
Rondale Moore
497
Skyy Moore
498
Joe Morgan
499
Josh Morgan
500
Stanley Morgan Jr.
501
Randy Moss
502
Santana Moss
503
Derek Moye
504
Keith Mumphery
505
Louis Murphy
506
Tyler Murphy
507
Legedu Naanee
508
Jalen Nailor
509
Jojo Natson
510
David Nelson
511
JJ Nelson
512
Jordy Nelson
513
Dazz Newsome
514
Jamar Newsome
515
Hakeem Nicks
516
Jordan Norwood
517
Kevin Norwood
518
Ben Obomanu
519
Kevin Ogletree
520
Chris Olave
521
Gun

yay
1
2
3
4
5


<ipython-input-26-96230670df94>:441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Week'] = test_df['Week'].astype(int)
<ipython-input-26-96230670df94>:442: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Year'] = test_df['Year'].astype(int)
<ipython-input-26-96230670df94>:452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

6
7
8
9
10
11
12


<ipython-input-26-96230670df94>:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[week_num_wks_ago] = np.where((test_df[curr_wk] == 1) & (test_df[curr_yr] >= 2022), 18,
<ipython-input-26-96230670df94>:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
<ipython-input-26-96230670df94>:466: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
          player_wk_year  week num 1 wks ago  year 1 wks ago  \
0   Davante Adams 5 2014                   4            2014   
1   Davante Adams 6 2014                   5            2014   
2   Davante Adams 7 2014                   6            2014   
3   Davante Adams 8 2014                   7            2014   
4  Davante Adams 11 2014                  10            2014   

        player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0   Davante Adams 4 2014                   3            2014   
1   Davante Adams 5 2014                   4            2014   
2   Davante Adams 6 2014                   5            2014   
3   Davante Adams 7 2014                   6            2014   
4  Davante Adams 10 2014                   9            2014   

       player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0  Davante Adams 3 2014                   2            2014   
1  Davante Adams 4 2014          

<ipython-input-26-96230670df94>:658: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-26-96230670df94>:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-26-96230670df94>:660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [41]:
wr_test2 = wr_test
wr_test2['Year'] = wr_test2['Year'].astype(int)
wr_test2 = wr_test2[wr_test2['Year']>2015]

In [42]:
wr_test2['Year'].value_counts()

2021    1278
2019    1221
2018    1219
2016    1216
2020    1212
2017    1152
2022    1007
Name: Year, dtype: int64

In [ ]:
#wr_test2

In [43]:
wr_test_final = create_qb_trend_var(qb_df=wr_test2,
                                 position = 'WR',
                                file_save = 'wr_player_data_final.csv')

(8305, 2586)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(8305, 3070)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 2
(8305, 4522)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(8305, 5006)


In [44]:
wr_test_final.shape

(8305, 5006)

In [46]:
curr_wk_wr_df = create_curr_wk_oot_qb(nfl_sched = nfl_sched,
                                   year= 2022, 
                                   week = 15,
                                   player_df=test_fun,
                                   position = 'WR',
                                  def_df = def_weekly,
                                  save_file = '2022wrweek15.csv')

0
Davante Adams
1
Nelson Agholor
2
Brandon Aiyuk
3
Maurice Alexander
4
Keenan Allen
5
Robbie Anderson
6
Dan Arnold
7
Tutu Atwell
8
Andre Baccellia
9
Michael Bandy
10
Rashod Bateman
11
Cole Beasley
12
David Bell
13
Trinity Benson
14
Jared Bernhardt
15
Braxton Berrios
16
Stanley Berryhill
17
C.J. Board
18
Kendrick Bourne
19
Lynn Bowden Jr.
20
Tyler Boyd
21
Miles Boykin
22
A.J. Brown
23
Dyami Brown
24
John Brown
25
Marquise Brown
26
Noah Brown
27
Treylon Burks
28
Damiere Byrd
29
Lawrence Cager
30
Marquez Callaway
31
Jalen Camp
32
Parris Campbell
33
DeAndre Carter
34
Quintez Cephus
35
DJ Chark
36
Ja'Marr Chase
37
Dan Chisena
38
Chase Claypool
39
Tyrie Cleveland
40
Randall Cobb
41
Keelan Cole
42
Nico Collins
43
Chris Conley
44
Brandin Cooks
45
Amari Cooper
46
Pharoh Cooper
47
Keke Coutee
48
Britain Covey
49
River Cracraft
50
Jamison Crowder
51
Frank Darby
52
Jaelon Darden
53
Corey Davis
54
Gabriel Davis
55
Stefon Diggs
56
Phillip Dorsett
57
Greg Dortch
58
Jahan Dotson
59
Romeo Doubs
60
Asht

215
Ben Edwards
216
Braylon Edwards
217
Bryan Edwards
218
Patrick Edwards
219
Bruce Ellington
220
Kevin Elliott
221
Quincy Enunwa
222
Alex Erickson
223
D'Wayne Eskridge
224
Mike Evans
225
Simi Fehoko
226
Ethan Fernea
227
Maurice Ffrench
228
Larry Fitzgerald
229
Dez Fitzpatrick
230
Malcom Floyd
231
Michael Floyd
232
Isaiah Ford
233
Jacoby Ford
234
Robert Foster
235
Daurice Fountain
236
Bennie Fowler
237
Mose Frazier
238
Isaac Fruechte
239
Travis Fulgham
240
Aaron Fuller
241
Corey Fuller
242
Will Fuller
243
Devin Funchess
244
Taylor Gabriel
245
Jabar Gaffney
246
Rico Gafford
247
Russell Gage
248
Michael Gallup
249
Antonio Gandy-Golden
250
Pierre Garcon
251
Clyde Gates
252
Kaylon Geiger
253
Tanner Gentry
254
David Gettis
255
Brandon Gibson
256
Shelton Gibson
257
David Gilreath
258
Mardy Gilyard
259
Ted Ginn Jr.
260
Chris Givens
261
Chris Godwin
262
Terry Godwin
263
Brittan Golden
264
Kenny Golladay
265
Richard Goodman
266
C.J. Goodwin
267
Marquise Goodwin
268
Josh Gordon
269
T.J. Graham
2

668
Maurice Stovall
669
Mike Strachan
670
Rod Streater
671
Devin Street
672
Tommy Streeter
673
Jaelen Strong
674
Sammie Stroughter
675
Courtland Sutton
676
Freddie Swain
677
Ryan Switzer
678
Julian Talley
679
Auden Tate
680
Brandon Tate
681
Golden Tate
682
Jordan Taylor
683
Kerry Taylor
684
Malik Taylor
685
Taywan Taylor
686
Trent Taylor
687
Adam Thielen
688
Marcus Thigpen
689
Demaryius Thomas
690
Lamaar Thomas
691
Michael Thomas
692
Mike Thomas
693
Mike Thomas
694
Deven Thompkins
695
Kenbrell Thompkins
696
Chris Thompson
697
Cody Thompson
698
Deonte Thompson
699
Tyquan Thornton
700
Jalen Tolbert
701
Jalen Tolliver
702
Kadarius Toney
703
Nick Toon
704
Samori Toure
705
Austin Trammell
706
Laquon Treadwell
707
Bryce Treggs
708
DJ Turner
709
Malik Turner
710
Patrick Turner
711
Paul Turner
712
KaVontae Turpin
713
Andrew Turzilli
714
Brian Tyms
715
Tiquan Underwood
716
John Ursua
717
Marquez Valdes-Scantling
718
Jordan Veasy
719
Justin Veltung
720
Binjimen Victor
721
Jalen Virgil
722
Jaylen

<ipython-input-28-1f29e38a0d97>:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['Week']
<ipython-input-28-1f29e38a0d97>:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['Year']
<ipython-input-28-1f29e38a0d97>:513: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

0
          player_wk_year  week num 1 wks ago  year 1 wks ago  \
0  Davante Adams 15 2022                  14            2022   
1   Randall Cobb 15 2022                  14            2022   
2    Romeo Doubs 15 2022                  14            2022   
3   Jake Kumerow 15 2022                  14            2022   
4  Allen Lazard  15 2022                  14            2022   

        player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0  Davante Adams 14 2022                  13            2022   
1   Randall Cobb 14 2022                  13            2022   
2    Romeo Doubs 14 2022                  13            2022   
3   Jake Kumerow 14 2022                  13            2022   
4  Allen Lazard  14 2022                  13            2022   

        player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0  Davante Adams 13 2022                  12            2022   
1   Randall Cobb 13 2022                  12            2022   
2    Romeo Doubs 13 2022            

<ipython-input-28-1f29e38a0d97>:715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-28-1f29e38a0d97>:716: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-28-1f29e38a0d97>:717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [47]:
curr_wk_wr_df.head(30)

,player_wk_year,week num 1 wks ago,year 1 wks ago,player 1 wks ago,week num 2 wks ago,year 2 wks ago,player 2 wks ago,week num 3 wks ago,year 3 wks ago,player 3 wks ago,...,def matchup DefTD 35wks ago,def matchup PA 35wks ago,def matchup PaYD 35wks ago,def matchup RuYd 35wks ago,def matchup Safety 35wks ago,def matchup KickTD 35wks ago,def matchup FPts 35wks ago,def matchup Opp 35wks ago,def matchup away flag 35wks ago,def matchup team_wk_year 35wks ago
0,Davante Adams 15 2022,14,2022,Davante Adams 14 2022,13,2022,Davante Adams 13 2022,12,2022,Davante Adams 12 2022,...,1,3,146.0,107.0,0,0,14.0,NE,0,LAR 14 2020
1,Davante Adams 15 2022,14,2022,Davante Adams 14 2022,13,2022,Davante Adams 13 2022,12,2022,Davante Adams 12 2022,...,0,24,137.0,186.0,0,0,3.0,LAR,1,NE 14 2020
2,Randall Cobb 15 2022,14,2022,Randall Cobb 14 2022,13,2022,Randall Cobb 13 2022,12,2022,Randall Cobb 12 2022,...,1,3,146.0,107.0,0,0,14.0,NE,0,LAR 14 2020
3,Randall Cobb 15 2022,14,2022,Randall Cobb 14 2022,13,2022,Randall Cobb 13 2022,12,2022,Randall Cobb 12 2022,...,0,31,212.0,249.0,0,0,3.0,TEN,0,JAX 14 2020
4,Randall Cobb 15 2022,14,2022,Randall Cobb 14 2022,13,2022,Randall Cobb 13 2022,12,2022,Randall Cobb 12 2022,...,0,27,316.0,80.0,1,1,14.0,MIA,1,KC 14 2020
5,Romeo Doubs 15 2022,14,2022,Romeo Doubs 14 2022,13,2022,Romeo Doubs 13 2022,12,2022,Romeo Doubs 12 2022,...,1,3,146.0,107.0,0,0,14.0,NE,0,LAR 14 2020
6,Jake Kumerow 15 2022,14,2022,Jake Kumerow 14 2022,13,2022,Jake Kumerow 13 2022,12,2022,Jake Kumerow 12 2022,...,1,3,146.0,107.0,0,0,14.0,NE,0,LAR 14 2020
7,Jake Kumerow 15 2022,14,2022,Jake Kumerow 14 2022,13,2022,Jake Kumerow 13 2022,12,2022,Jake Kumerow 12 2022,...,0,20,243.0,105.0,0,0,3.0,LAC,1,ATL 14 2020
8,Jake Kumerow 15 2022,14,2022,Jake Kumerow 14 2022,13,2022,Jake Kumerow 13 2022,12,2022,Jake Kumerow 12 2022,...,0,33,393.0,94.0,0,0,11.0,KC,0,MIA 14 2020
9,Jake Kumerow 15 2022,14,2022,Jake Kumerow 14 2022,13,2022,Jake Kumerow 13 2022,12,2022,Jake Kumerow 12 2022,...,0,14,225.0,162.0,0,0,8.0,MIN,0,TB 14 2020


In [ ]:
#curr_wk_wr_df['temp_col'] = curr_wk_wr_df['CITY'] + " 5 2022"
#curr_wk_wr_df = curr_wk_wr_df[curr_wk_wr_df['temp_col'] == curr_wk_wr_df['opp wk year']]
#curr_wk_wr_df = curr_wk_wr_df.drop(columns = ['temp_col'])
#curr_wk_wr_df = curr_wk_wr_df.drop_duplicates()

In [48]:
wr_wk15_final = create_qb_trend_var(qb_df=curr_wk_wr_df,
                                 position = 'WR',
                                file_save = '2022wrweek15_final.csv')

(466, 2515)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(466, 2999)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 2
(466, 4451)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(1260, 4935)


In [ ]:
#wr_temp = wr_wk4_final.drop_duplicates(subset = 'player_wk_year', keep='first')

In [ ]:
#wr_temp.shape

In [ ]:
#wr_temp.to_csv('2022wrweek4_final.csv')

In [49]:
te_test = position_feat_eng_df_creation(player_df = test_fun,
                              def_df = test,
                              position = 'TE',
                                     file_save = 'te_player_data_v1.csv')

0
Jerell Adams
1
Kyle Adams
2
Jordan Akins
3
Mo Alie-Cox
4
Dwayne Allen
5
Rashaun Allen
6
Jace Amaro
7
Stephen Anderson
8
Mark Andrews
9
Blake Annen
10
Devin Asiasi
11
Antony Auclair
12
Kennard Backman
13
Billy Bajema
14
Jake Ballard
15
Brandon Barden
16
Gary Barnidge
17
John Bates
18
Marcus Baugh
19
Alex Bayer
20
Evan Baylis
21
Andrew Beck
22
Nate Becker
23
Travis Beckum
24
Blake Bell
25
Daniel Bellinger
26
Martellus Bennett
27
Rashod Berry
28
E.J. Bibbs
29
Kendall Blanton
30
Kevin Boss
31
Brandon Bostick
32
Nick Bowers
33
Braedon Bowman
34
Nick Boyle
35
Brett Brackett
36
Cameron Brate
37
Ben Braunecker
38
Kevin Brock
39
Richie Brockel
40
Daniel Brown
41
Pharaoh Brown
42
Harrison Bryant
43
Hunter Bryant
44
Ian Bunting
45
Trey Burton
46
Matt Bushman
47
Jake Butt
48
Nate Byham
49
Emanuel Byrd
50
Jake Byrne
51
Grant Calcaterra
52
Jordan Cameron
53
John Carlson
54
Stephen Carlson
55
Cethan Carter
56
Kyle Carter
57
Roger Carter
58
James Casey
59
Brent Celek
60
Garrett Celek
61
Scott Chandl

<ipython-input-26-96230670df94>:441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Week'] = test_df['Week'].astype(int)
<ipython-input-26-96230670df94>:442: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Year'] = test_df['Year'].astype(int)
<ipython-input-26-96230670df94>:452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

3
4
5
6
7
8
9
10
11


<ipython-input-26-96230670df94>:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[year_wks_ago] = np.where(test_df[curr_wk] == 1, test_df[curr_yr]-1, test_df[curr_yr])
<ipython-input-26-96230670df94>:466: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[player_wks_ago] = test_df['Player '] + " " + test_df[week_num_wks_ago].astype(str) + " " + test_df[year_wks_ago].astype(str)
<ipython-input-26-96230670df94>:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
0
         player_wk_year  week num 1 wks ago  year 1 wks ago  \
0  Jerell Adams 15 2016                  14            2016   
1  Jerell Adams 16 2016                  15            2016   
2  Jerell Adams 10 2017                   9            2017   
3     Kyle Adams 4 2012                   3            2012   
4     Kyle Adams 8 2012                   7            2012   

       player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0  Jerell Adams 14 2016                  13            2016   
1  Jerell Adams 15 2016                  14            2016   
2   Jerell Adams 9 2017                   8            2017   
3     Kyle Adams 3 2012                   2            2012   
4     Kyle Adams 7 2012                   6            2012   

       player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0  Jerell Adams 13 2016                  12            2016   
1  Jerell Adams 14 2016                  1

<ipython-input-26-96230670df94>:658: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-26-96230670df94>:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-26-96230670df94>:660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [50]:
te_test2 = te_test
te_test2['Year'] = te_test2['Year'].astype(int)
te_test2 = te_test2[te_test2['Year']>2015]

In [ ]:
#te_test2 = te_test2.drop_duplicates(subset = 'player_wk_year', keep='first')

In [51]:
te_test2.shape

(4852, 2761)

In [52]:
te_test_final = create_qb_trend_var(qb_df=te_test2,
                                 position = 'TE',
                                file_save = 'te_player_data_final.csv')

(4852, 2586)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(4852, 3070)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

<ipython-input-27-2de3c4070737>:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2','new col 3',
<ipython-input-27-2de3c4070737>:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_min] = temp_df2[['new col 1', 'new col 2','new col 3',
<ipython-input-27-2de3c4070737>:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

merge 2
(4852, 4522)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(4852, 5006)


In [53]:
curr_wk_te_df = create_curr_wk_oot_qb(nfl_sched = nfl_sched,
                                   year= 2022, 
                                   week = 15,
                                   player_df=test_fun,
                                   position = 'TE',
                                  def_df = def_weekly,
                                  save_file = '2022teweek15.csv')

0
Jordan Akins
1
Mo Alie-Cox
2
Stephen Anderson
3
Mark Andrews
4
Devin Asiasi
5
John Bates
6
Andrew Beck
7
Daniel Bellinger
8
Kendall Blanton
9
Nick Boyle
10
Cameron Brate
11
Pharaoh Brown
12
Harrison Bryant
13
Grant Calcaterra
14
Cethan Carter
15
Roger Carter
16
Tyler Conklin
17
Tanner Conner
18
Tyler Davis
19
Josiah Deguara
20
Will Dissly
21
Greg Dulcich
22
Ross Dwelley
23
Ben Ellefson
24
Evan Engram
25
Zach Ertz
26
Gerald Everett
27
Noah Fant
28
Luke Farrell
29
Jake Ferguson
30
Anthony Firkser
31
Miller Forristall
32
Jody Fortson
33
Pat Freiermuth
34
Zach Gentry
35
Mike Gesicki
36
Reggie Gilliam
37
Dallas Goedert
38
Kylen Granson
39
Noah Gray
40
Ryan Griffin
41
Clark Harris
42
Peyton Hendershot
43
Hunter Henry
44
Parker Hesse
45
Connor Heyward
46
Tyler Higbee
47
T.J. Hockenson
48
Jacob Hollister
49
J.P. Holtz
50
Austin Hooper
51
Brycen Hopkins
52
O.J. Howard
53
Tanner Hudson
54
Hayden Hurst
55
Tyree Jackson
56
Jesse James
57
Brevin Jordan
58
Nikola Kalinic
59
Travis Kelce
60
Ko Kief

348
Jack Stoll
349
Jake Stoneburner
350
Zach Sudfeld
351
Stephen Sullivan
352
Phillip Supernaw
353
Geoff Swaim
354
Tommy Sweeney
355
Erik Swoope
356
Tyrone Swoopes
357
Jacob Tamme
358
Ryan Taylor
359
Randall Telfer
360
David Thomas
361
Ian Thomas
362
Jordan Thomas
363
Julius Thomas
364
Colin Thompson
365
Jordan Thompson
366
Taylor Thompson
367
D.J. Tialavea
368
Noah Togiai
369
Levine Toilolo
370
Eric Tomlinson
371
Jake Tonges
372
Robert Tonyan
373
Adam Trautman
374
Ross Travis
375
Austin Traylor
376
Tommy Tremble
377
Cole Turner
378
Will Tye
379
C.J. Uzomah
380
Hakeem Valles
381
Jason Vander Laan
382
Nick Vannett
383
Matt Veldman
384
Clive Walford
385
Delanie Walker
386
Darren Waller
387
Kahale Warring
388
Ben Watson
389
David Wells
390
Trevon Wesco
391
Cole Wick
392
Mitchell Wilcox
393
Brandon Williams
394
D.J. Williams
395
Maxx Williams
396
Michael Williams
397
Luke Willson
398
Caleb Wilson
399
Kellen Winslow
400
Jason Witten
401
Charlie Woerner
402
Ethan Wolf
403
Jelani Woods
404
Br

<ipython-input-28-1f29e38a0d97>:511: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['Week']
<ipython-input-28-1f29e38a0d97>:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['Year']
<ipython-input-28-1f29e38a0d97>:513: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

0
             player_wk_year  week num 1 wks ago  year 1 wks ago  \
0      Jordan Akins 15 2022                  14            2022   
1  Stephen Anderson 15 2022                  14            2022   
2     Pharaoh Brown 15 2022                  14            2022   
3      Ryan Griffin 15 2022                  14            2022   
4      Clark Harris 15 2022                  14            2022   

           player 1 wks ago  week num 2 wks ago  year 2 wks ago  \
0      Jordan Akins 14 2022                  13            2022   
1  Stephen Anderson 14 2022                  13            2022   
2     Pharaoh Brown 14 2022                  13            2022   
3      Ryan Griffin 14 2022                  13            2022   
4      Clark Harris 14 2022                  13            2022   

           player 2 wks ago  week num 3 wks ago  year 3 wks ago  \
0      Jordan Akins 13 2022                  12            2022   
1  Stephen Anderson 13 2022                  12           

<ipython-input-28-1f29e38a0d97>:715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-28-1f29e38a0d97>:716: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-28-1f29e38a0d97>:717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013                    1            2013   
2    

In [ ]:
#curr_wk_te_df

In [ ]:
#curr_wk_te_df[['player_wk_year','CITY','opp wk year','team_wk_year']]

In [ ]:
#curr_wk_te_df['temp_col'] = curr_wk_te_df['CITY'] + " 5 2022"
#curr_wk_te_df = curr_wk_te_df[curr_wk_te_df['temp_col'] == curr_wk_te_df['opp wk year']]
#curr_wk_te_df = curr_wk_te_df.drop(columns = ['temp_col'])
#curr_wk_te_df = curr_wk_te_df.drop_duplicates()

In [54]:
te_wk15_final = create_qb_trend_var(qb_df=curr_wk_te_df,
                              position = 'TE',
                                file_save = '2022teweek15_final.csv')

(214, 2515)
trend 1 creation


<ipython-input-27-2de3c4070737>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

merge 1
(214, 2999)
trend 2 creation


<ipython-input-27-2de3c4070737>:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 2
(214, 4451)
trend 3 creation


<ipython-input-27-2de3c4070737>:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:526: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-27-2de3c4070737>:527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

merge 3
(494, 4935)


In [ ]:
#te_temp = te_wk4_final.drop_duplicates(subset = 'player_wk_year', keep='first')

In [ ]:
#te_temp.shape

In [ ]:
#te_temp.to_csv('2022teweek4_final.csv')

In [55]:
def_temp = def_feat_eng_df_creation(def_df = test,
                                 file_save = 'def_team_data.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


<ipython-input-29-e37dc8bfefd1>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-29-e37dc8bfefd1>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-29-e37dc8bfefd1>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

25
26
27
28
29
30
31
32
33
34
35
0
       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013           

<ipython-input-29-e37dc8bfefd1>:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-29-e37dc8bfefd1>:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-29-e37dc8bfefd1>:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
test
0
       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  opp_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0             LAR 3 2013                   2            2013   
1             JAX 3 2013                   2            2013   
2             SEA 3 2013                   2            2013   
3              TB 3 2013                   2            2013   
4             LAC 3 2013                   2            2013   

   opp_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0              LAR 2 2013                    1            2013  

<ipython-input-29-e37dc8bfefd1>:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

<ipython-input-29-e37dc8bfefd1>:484: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [56]:
def_temp.tail()

,Week,FPts,week_num,year,Opp,team_wk_year,opp_wk_year,team_wk_year 1 wks ago,team_wk_year 2 wks ago,team_wk_year 3 wks ago,...,min def matchup away flag 18wk trend,max def matchup away flag 18wk trend,sum def matchup away flag 24wk trend,avg def matchup away flag 24wk trend,min def matchup away flag 24wk trend,max def matchup away flag 24wk trend,sum def matchup away flag 30wk trend,avg def matchup away flag 30wk trend,min def matchup away flag 30wk trend,max def matchup away flag 30wk trend
4027,14,2.0,14,2022,CAR,SEA 14 2022,CAR 14 2022,SEA 13 2022,SEA 12 2022,SEA 11 2022,...,0.0,1.0,12.0,0.500000,0.0,1.0,15.0,0.500000,0.0,1.0
4028,14,2.0,14,2022,MIA,LAC 14 2022,MIA 14 2022,LAC 13 2022,LAC 12 2022,LAC 11 2022,...,0.0,1.0,12.0,0.500000,0.0,1.0,14.0,0.466667,0.0,1.0
4029,14,2.0,14,2022,SF,TB 14 2022,SF 14 2022,TB 13 2022,TB 12 2022,TB 11 2022,...,0.0,1.0,11.0,0.458333,0.0,1.0,14.0,0.466667,0.0,1.0
4030,14,0.0,14,2022,JAX,TEN 14 2022,JAX 14 2022,TEN 13 2022,TEN 12 2022,TEN 11 2022,...,0.0,1.0,12.0,0.500000,0.0,1.0,15.0,0.500000,0.0,1.0
4031,14,0.0,14,2022,DET,MIN 14 2022,DET 14 2022,MIN 13 2022,MIN 12 2022,MIN 11 2022,...,0.0,1.0,11.0,0.458333,0.0,1.0,13.0,0.433333,0.0,1.0


In [57]:
xx = def_curr_wk_oot(def_df = test,
                     nfl_sched = nfl_sched, 
                     curr_wk = 15, 
                     curr_yr = 2022,
                    file_save = '2022defwk15_final.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


<ipython-input-29-e37dc8bfefd1>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-29-e37dc8bfefd1>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-29-e37dc8bfefd1>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

26
27
28
29
30
31
32
33
34
35
0
       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  team_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0              SEA 3 2013                   2            2013   
1              CAR 3 2013                   2            2013   
2              LAR 3 2013                   2            2013   
3              TEN 3 2013                   2            2013   
4              DET 3 2013                   2            2013   

   team_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0               SEA 2 2013                    1            2013   
1               CAR 2 2013              

<ipython-input-29-e37dc8bfefd1>:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_wk] = test_df['week_num']
<ipython-input-29-e37dc8bfefd1>:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[curr_yr] = test_df['year']
<ipython-input-29-e37dc8bfefd1>:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
test
0
       Team CITY  Week Sack FR INT DefTD  PA   PaYD   RuYd  ...  \
0  Seahawks  SEA     1    2  3   0     1  34  297.0   76.0  ...   
1  Panthers  CAR     1    5  1   2     1   9  183.0   96.0  ...   
2      Rams  LAR     1    6  0   1     0  31  251.0  124.0  ...   
3    Titans  TEN     1    4  0   2     1  14  210.0   92.0  ...   
4     Lions  DET     1    2  1   2     1  33  404.0   95.0  ...   

  opp_wk_year 32 wks ago week num 33 wks ago year 33 wks ago  \
0             LAR 3 2013                   2            2013   
1             JAX 3 2013                   2            2013   
2             SEA 3 2013                   2            2013   
3              TB 3 2013                   2            2013   
4             LAC 3 2013                   2            2013   

   opp_wk_year 33 wks ago  week num 34 wks ago year 34 wks ago  \
0              LAR 2 2013                    1   

<ipython-input-29-e37dc8bfefd1>:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

<ipython-input-29-e37dc8bfefd1>:484: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_sum] = temp_df2[['new col 1', 'new col 2']].sum(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[newcol_avg] = temp_df2[['new col 1', 'new col 2']].mean(skipna= False, axis = 1)
<ipython-input-29-e37dc8bfefd1>:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [58]:
xx.to_csv('2022defwk15_final.csv')